# This module is used to handle below data cleaning on the flattened restaurant data
   * Identify missing values
   * Replace missing values with relevant values
   * Convert Alphanumeric categorical values to Numerical values
   * Clean data with typos and bring in uniformity of values based on analysis in Tableau.

Importing the required packages

In [1]:
import pandas as pd
import numpy as np
import os
import json
import re

Setting the display options

In [2]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

Reading the flattened csv file into a dataframe for EDA 

In [3]:
businessDF= pd.read_csv('AZ_Restaurant_Final_data.csv', sep = ';', header = 'infer', index_col ='Unnamed: 0')

C:\Users\nishi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (62,63,64,65,66,67,68,69) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Displaying some values of the dataframe to understand the layout

In [4]:
businessDF.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,AcceptsInsurance,AgesAllowed,Alcohol,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,Ambience_upscale,BestNights_friday,BestNights_monday,BestNights_saturday,BestNights_sunday,BestNights_thursday,BestNights_tuesday,BestNights_wednesday,BikeParking,BusinessAcceptsBitcoin,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_lot,BusinessParking_street,BusinessParking_valet,BusinessParking_validated,ByAppointmentOnly,BYOB,BYOBCorkage,Caters,CoatCheck,Corkage,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_halal,DietaryRestrictions_kosher,DietaryRestrictions_soy-free,DietaryRestrictions_vegan,DietaryRestrictions_vegetarian,DogsAllowed,DriveThru,GoodForDancing,GoodForKids,GoodForMeal_breakfast,GoodForMeal_brunch,GoodForMeal_dessert,GoodForMeal_dinner,GoodForMeal_latenight,GoodForMeal_lunch,HairSpecializesIn_africanamerican,HairSpecializesIn_asian,HairSpecializesIn_coloring,HairSpecializesIn_curly,HairSpecializesIn_extensions,HairSpecializesIn_kids,HairSpecializesIn_perms,HairSpecializesIn_straightperms,HappyHour,HasTV,Music_background_music,Music_dj,Music_jukebox,Music_karaoke,Music_live,Music_no_music,Music_video,NoiseLevel,Open24Hours,OutdoorSeating,RestaurantsAttire,RestaurantsCounterService,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,category_Afghan,category_African,category_American (New),category_American (Traditional),category_Arabian,category_Arcades,category_Argentine,category_Armenian,category_Arts & Entertainment,category_Asian Fusion,category_Australian,category_Austrian,category_Bagels,category_Bakeries,category_Barbeque,category_Bars,category_Beer Bar,category_Beer Gardens,category_Beer,category_Belgian,category_Bistros,category_Brasseries,category_Brazilian,category_Breakfast & Brunch,category_Breweries,category_Brewpubs,category_British,category_Bubble Tea,category_Buffets,category_Burgers,category_Cafes,category_Cafeteria,category_Cajun/Creole,category_Cambodian,category_Cantonese,category_Caribbean,category_Casinos,category_Caterers,category_Cheesesteaks,category_Chicken Shop,category_Chicken Wings,category_Chinese,category_Chocolatiers & Shops,category_Cocktail Bars,category_Coffee & Tea Supplies,category_Coffee & Tea,category_Coffee Roasteries,category_Coffeeshops,category_Colombian,category_Comfort Food,category_Creperies,category_Cuban,category_Cupcakes,category_Custom Cakes,category_Czech,category_Delis,category_Desserts,category_Dim Sum,category_Diners,category_Dinner Theater,category_Do-It-Yourself Food,category_Dominican,category_Donuts,category_Eatertainment,category_Ethiopian,category_Ethnic Food,category_Event Planning & Services,category_Falafel,category_Fast Food,category_Festivals,category_Filipino,category_Fish & Chips,category_Fondue,category_Food Court,category_Food Delivery Services,category_Food Stands,category_Food Trucks,category_Food,category_French,category_Gastropubs,category_German,category_Gluten-Free,category_Greek,category_Grocery,category_Halal,category_Hawaiian,category_Herbs & Spices,category_Hookah Bars,category_Hot Dogs,category_Hot Pot,category_Hotel bar,category_Hotels,category_Ice Cream & Frozen Yogurt,category_Imported Food,category_Indian,category_Indonesian,category_Irish Pub,category_Irish,category_Italian,category_Izakaya,category_Japanese,category_Juice Bars & Smoothies,category_Kebab,category_Korean,category_Kosher,category_Laotian,category_Latin American,category_Lebanese,category_Live/Raw Food,category_Local Flavor,category_Lounges,category_Macarons,category_Malaysian,category_Marinas,category_Meat Shops,category_Mediterranean,category_Mexican,category_Middle Eastern,category_Modern European,category_Mongolian,category

Determing the number of rows and columns of the dataframe

In [5]:
businessDF.shape

(7367, 276)

Checking the data types of the various columns in the dataframe

In [6]:
businessDF.dtypes

business_id                            object
name                                   object
neighborhood                          float64
address                                object
city                                   object
state                                  object
postal_code                           float64
latitude                              float64
longitude                             float64
stars                                 float64
review_count                            int64
AcceptsInsurance                       object
AgesAllowed                            object
Alcohol                                object
Ambience_casual                        object
Ambience_classy                        object
Ambience_divey                         object
Ambience_hipster                       object
Ambience_intimate                      object
Ambience_romantic                      object
Ambience_touristy                      object
Ambience_trendy                   

Taking a back up copy of the dataframe for reference.

In [7]:
businessDF1=businessDF.copy(deep=True)

Cross verifying the columns and the dataframe structure.

In [8]:
businessDF.columns.values

array(['business_id', 'name', 'neighborhood', 'address', 'city', 'state',
       'postal_code', 'latitude', 'longitude', 'stars', 'review_count',
       'AcceptsInsurance', 'AgesAllowed', 'Alcohol', 'Ambience_casual',
       'Ambience_classy', 'Ambience_divey', 'Ambience_hipster',
       'Ambience_intimate', 'Ambience_romantic', 'Ambience_touristy',
       'Ambience_trendy', 'Ambience_upscale', 'BestNights_friday',
       'BestNights_monday', 'BestNights_saturday', 'BestNights_sunday',
       'BestNights_thursday', 'BestNights_tuesday',
       'BestNights_wednesday', 'BikeParking', 'BusinessAcceptsBitcoin',
       'BusinessAcceptsCreditCards', 'BusinessParking_garage',
       'BusinessParking_lot', 'BusinessParking_street',
       'BusinessParking_valet', 'BusinessParking_validated',
       'ByAppointmentOnly', 'BYOB', 'BYOBCorkage', 'Caters', 'CoatCheck',
       'Corkage', 'DietaryRestrictions_dairy-free',
       'DietaryRestrictions_gluten-free', 'DietaryRestrictions_halal',
       '

Determing the overall number of null values for each column of the dataframe

In [9]:
businessDF.isnull().sum(axis=0)

business_id                              0
name                                     0
neighborhood                          7367
address                                 86
city                                     0
state                                    0
postal_code                              9
latitude                                 0
longitude                                0
stars                                    0
review_count                             0
AcceptsInsurance                      7362
AgesAllowed                           7354
Alcohol                               1074
Ambience_casual                        869
Ambience_classy                        869
Ambience_divey                         869
Ambience_hipster                       869
Ambience_intimate                      869
Ambience_romantic                      869
Ambience_touristy                      869
Ambience_trendy                        869
Ambience_upscale                       869
BestNights_

In [10]:
len(businessDF) - businessDF.count()

business_id                              0
name                                     0
neighborhood                          7367
address                                 86
city                                     0
state                                    0
postal_code                              9
latitude                                 0
longitude                                0
stars                                    0
review_count                             0
AcceptsInsurance                      7362
AgesAllowed                           7354
Alcohol                               1074
Ambience_casual                        869
Ambience_classy                        869
Ambience_divey                         869
Ambience_hipster                       869
Ambience_intimate                      869
Ambience_romantic                      869
Ambience_touristy                      869
Ambience_trendy                        869
Ambience_upscale                       869
BestNights_

The below mentioned loop will print the number of NaN values, sub categories and the value counts for all the columns of the dataframe along with the column details. This will help us to group and handle the NaN values of the columns in the dataframe.

In [11]:
for colmn in businessDF.columns.values:
    print('Column name: ',colmn)
    print('No of Unique values: ',businessDF[colmn].nunique())
    print('Unique values: ',businessDF[colmn].unique())
    print('NaN count: ',businessDF[colmn].isnull().sum())
    if(colmn not in ['business_id','name','neighborhood','address','city','state','postal_code','latitude','longitude',
                     'stars','review_count']):
        print('Attribute counts:\n',businessDF[colmn].value_counts(),'\n')
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

Column name:  business_id
No of Unique values:  7367
Unique values:  ['rDMptJYWtnMhpQu_rRXHng' '1WBkAuQg81kokZIPMpn9Zg'
 'iPa__LOhse-hobC2Xmp-Kw' ... '5zva2MTtB5IX6TaoVLL-NA'
 'Gr-2oBg4XyduSKbvnE-i9g' 'UdEmYOnk2iJDY9lpEPAlJQ']
NaN count:  0
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  name
No of Unique values:  4172
Unique values:  ["McDonald's" 'Charr An American Burger Bar' 'Little Caesars Pizza' ...
 "Yin's Chinese Resturant" 'Zorbas Grill'
 'Salt & Lime Modern Mexican Grill']
NaN count:  0
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  neighborhood
No of Unique values:  0
Unique values:  [nan]
NaN count:  7367
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  address
No of Unique values:  6721
Unique values:  ['719 E Thunderbird Rd' '777 E Thunderbird Rd, Ste 107'
 '1635 E Camelback Rd' ... '1950 W Indian School Rd'
 '9397 E Shea Blvd, Ste 115' '5

Unique values:  ['free' 'paid' nan 'no']
NaN count:  1232
Attribute counts:
 no      3294
free    2797
paid      44
Name: WiFi, dtype: int64 

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  category_Afghan
No of Unique values:  1
Unique values:  [nan  1.]
NaN count:  7363
Attribute counts:
 1.0    4
Name: category_Afghan, dtype: int64 

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  category_African
No of Unique values:  1
Unique values:  [nan  1.]
NaN count:  7357
Attribute counts:
 1.0    10
Name: category_African, dtype: int64 

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  category_American (New)
No of Unique values:  1
Unique values:  [nan  1.]
NaN count:  6618
Attribute counts:
 1.0    749
Name: category_American (New), dtype: int64 

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  category_American

Column name:  category_Modern European
No of Unique values:  1
Unique values:  [nan  1.]
NaN count:  7358
Attribute counts:
 1.0    9
Name: category_Modern European, dtype: int64 

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  category_Mongolian
No of Unique values:  1
Unique values:  [nan  1.]
NaN count:  7357
Attribute counts:
 1.0    10
Name: category_Mongolian, dtype: int64 

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  category_Moroccan
No of Unique values:  1
Unique values:  [nan  1.]
NaN count:  7363
Attribute counts:
 1.0    4
Name: category_Moroccan, dtype: int64 

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  category_Music Venues
No of Unique values:  1
Unique values:  [nan  1.]
NaN count:  7323
Attribute counts:
 1.0    44
Name: category_Music Venues, dtype: int64 

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

From the above list we can find that the below mentioned values for the variable 'x' are either not relevant to the restaurant dataframe or have another column having similar details.

In [12]:
for x in ['neighborhood','AcceptsInsurance','BestNights_friday','BestNights_monday','BestNights_saturday',
 'BestNights_sunday','BestNights_thursday','BestNights_tuesday','BestNights_wednesday','BusinessAcceptsBitcoin',
 'ByAppointmentOnly','BYOB','BYOBCorkage','Caters','CoatCheck','Corkage','GoodForDancing','HairSpecializesIn_africanamerican',
 'HairSpecializesIn_asian','HairSpecializesIn_coloring','HairSpecializesIn_curly','HairSpecializesIn_extensions',
 'HairSpecializesIn_kids','HairSpecializesIn_perms','HairSpecializesIn_straightperms','Music_background_music',
 'Music_dj','Music_jukebox','Music_karaoke','Music_live','Music_video','Open24Hours','RestaurantsCounterService','category_Restaurants','hours_Monday',
 'hours_Tuesday','hours_Wednesday','hours_Thursday','hours_Friday','hours_Saturday','hours_Sunday']:
    businessDF.drop(x,axis=1,inplace=True)

Cross verifying the structure and the columns of the dataframe

In [13]:
businessDF.shape

(7367, 235)

NOTE: Other than the below mentioned columns, we are converting all other columns' values to either 0 or 1 if they are having only True or False value and using One hot enocding for all other kinds of values. This is done as it helps us to do the recommendation with ease.

'business_id','name','neighborhood','address','city','state','postal_code','latitude','longitude','stars','review_count'

## Handling missing values

#### AgesAllowed

Replacing missing values in 'AgesAllowed' with 'allages' as it includes '21plus' ages as well. Then converting it to 0's and 1's. Determining the value counts of the same.

In [14]:
businessDF['AgesAllowed']=businessDF['AgesAllowed'].fillna('allages')

In [15]:
businessDF['AgesAllowed'].unique()

array(['allages', '21plus'], dtype=object)

In [16]:
businessDF['AgesAllowed']=businessDF['AgesAllowed'].replace('21plus','1')
businessDF['AgesAllowed']=businessDF['AgesAllowed'].replace('allages','0')

In [17]:
businessDF['AgesAllowed'].value_counts()

0    7356
1      11
Name: AgesAllowed, dtype: int64

#### Alcohol

The 'Alcohol' columns's NaN values are converted to 'none' as they are assumed that the restaurant is not serving any alcohol.Then applying One hot encoding and determing the value counts of the same.

In [18]:
businessDF['Alcohol']=businessDF['Alcohol'].fillna('none')

In [19]:
businessDF['Alcohol'].unique()

array(['none', 'full_bar', 'beer_and_wine'], dtype=object)

In [20]:
businessDF['Alcohol']=businessDF['Alcohol'].replace('none','0')
businessDF['Alcohol']=businessDF['Alcohol'].replace('beer_and_wine','1')
businessDF['Alcohol']=businessDF['Alcohol'].replace('full_bar','2')

In [21]:
businessDF['Alcohol'].value_counts()

0    4560
2    1928
1     879
Name: Alcohol, dtype: int64

#### NoiseLevel

The 'NoiseLevel' having missing values is considered to be 'average' noise.

In [22]:
businessDF['NoiseLevel']=businessDF['NoiseLevel'].fillna('average')

In [23]:
businessDF['NoiseLevel'].unique()

array(['loud', 'average', 'quiet', 'very_loud'], dtype=object)

In [24]:
businessDF['NoiseLevel']=businessDF['NoiseLevel'].replace('quiet','0')
businessDF['NoiseLevel']=businessDF['NoiseLevel'].replace('average','1')
businessDF['NoiseLevel']=businessDF['NoiseLevel'].replace('loud','2')
businessDF['NoiseLevel']=businessDF['NoiseLevel'].replace('very_loud','3')

In [25]:
businessDF['NoiseLevel'].value_counts()

1    5739
0    1133
2     340
3     155
Name: NoiseLevel, dtype: int64

#### RestaurantsAttire

The 'RestaurantsAttire' is assumed to be casual if nothing is specified.

In [26]:
businessDF['RestaurantsAttire']=businessDF['RestaurantsAttire'].fillna('casual')

In [27]:
businessDF['RestaurantsAttire'].unique()

array(['casual', 'dressy', 'formal'], dtype=object)

In [28]:
businessDF['RestaurantsAttire']=businessDF['RestaurantsAttire'].replace('casual','0')
businessDF['RestaurantsAttire']=businessDF['RestaurantsAttire'].replace('dressy','1')
businessDF['RestaurantsAttire']=businessDF['RestaurantsAttire'].replace('formal','2')

In [29]:
businessDF['RestaurantsAttire'].value_counts()

0    7277
1      80
2      10
Name: RestaurantsAttire, dtype: int64

#### RestaurantsPriceRange2

The attribute 'RestaurantsPriceRange2' is used to rank the restaurants based on their prices for the food and drinks. If no price range is present it is considered to be 0.

In [30]:
businessDF['RestaurantsPriceRange2']=businessDF['RestaurantsPriceRange2'].fillna(0)

In [31]:
businessDF['RestaurantsPriceRange2'] = businessDF['RestaurantsPriceRange2'].astype(int)

In [32]:
businessDF['RestaurantsPriceRange2'].dtype

dtype('int32')

In [33]:
businessDF['RestaurantsPriceRange2'].unique()

array([1, 2, 0, 3, 4], dtype=int64)

#### Smoking

The 'Smoking' attribute's missing values is considered to be 'no', as it signifies that the restaurant is not providing either indoor or outdoor smoking area to the customer

In [34]:
businessDF['Smoking']=businessDF['Smoking'].fillna('no')

In [35]:
businessDF['Smoking'].unique()

array(['no', 'outdoor', 'yes'], dtype=object)

In [36]:
businessDF['Smoking']=businessDF['Smoking'].replace('no','0')
businessDF['Smoking']=businessDF['Smoking'].replace('outdoor','1')
businessDF['Smoking']=businessDF['Smoking'].replace('yes','2')

In [37]:
businessDF['Smoking'] = businessDF['Smoking'].astype(int)

In [38]:
businessDF['Smoking'].dtype

dtype('int32')

In [39]:
businessDF['Smoking'].value_counts()

0    6953
1     391
2      23
Name: Smoking, dtype: int64

#### WiFi

'WiFi' column in the dataframe having missing values is treated as a 'no', as it implies that there is not 'WiFi' facility available in the restaurant.

In [40]:
businessDF['WiFi']=businessDF['WiFi'].fillna('no')

In [41]:
businessDF['WiFi'].unique()

array(['free', 'paid', 'no'], dtype=object)

In [42]:
businessDF['WiFi']=businessDF['WiFi'].replace('no','0')
businessDF['WiFi']=businessDF['WiFi'].replace('free','1')
businessDF['WiFi']=businessDF['WiFi'].replace('paid','2')

In [43]:
businessDF['WiFi'].value_counts()

0    4526
1    2797
2      44
Name: WiFi, dtype: int64

#### Music_no_music

The attribute 'Music_no_music' indicates that there is no facility to play music in the restaurant if the value is True. Hence missing values are converted to 'True', followed by converting True to 0 and False to 1 respectively.

In [44]:
businessDF['Music_no_music']=businessDF['Music_no_music'].fillna('True')

In [45]:
businessDF['Music_no_music'].unique()

array(['True', False], dtype=object)

In [46]:
businessDF['Music_no_music']=businessDF['Music_no_music'].replace('True','0')
businessDF['Music_no_music']=businessDF['Music_no_music'].replace(False,'1')

In [47]:
businessDF['Music_no_music'].value_counts()

0    6533
1     834
Name: Music_no_music, dtype: int64

The below mentioned loop is for determining the columns having only either 0 or 1 to handle the missing values together.

In [48]:
list_cat=[]

In [49]:
for colmn in businessDF.columns.values:
    total=businessDF[colmn].nunique()
    if(total == 2 and colmn not in ['AgesAllowed','Alcohol','NoiseLevel','RestaurantsAttire','RestaurantsPriceRange2','Smoking',
 'WiFi']):
        list_cat.append(colmn)

In [50]:
len(list_cat)

42

There are 41 such columns now having only 0 and 1 as their values. Determining the column names below for reference.

In [51]:
for x in list_cat:
    print(x,'\t',type(x))

Ambience_casual 	 <class 'str'>
Ambience_classy 	 <class 'str'>
Ambience_divey 	 <class 'str'>
Ambience_hipster 	 <class 'str'>
Ambience_intimate 	 <class 'str'>
Ambience_romantic 	 <class 'str'>
Ambience_touristy 	 <class 'str'>
Ambience_trendy 	 <class 'str'>
Ambience_upscale 	 <class 'str'>
BikeParking 	 <class 'str'>
BusinessAcceptsCreditCards 	 <class 'str'>
BusinessParking_garage 	 <class 'str'>
BusinessParking_lot 	 <class 'str'>
BusinessParking_street 	 <class 'str'>
BusinessParking_valet 	 <class 'str'>
BusinessParking_validated 	 <class 'str'>
DietaryRestrictions_dairy-free 	 <class 'str'>
DietaryRestrictions_gluten-free 	 <class 'str'>
DietaryRestrictions_halal 	 <class 'str'>
DietaryRestrictions_kosher 	 <class 'str'>
DietaryRestrictions_soy-free 	 <class 'str'>
DietaryRestrictions_vegan 	 <class 'str'>
DietaryRestrictions_vegetarian 	 <class 'str'>
DogsAllowed 	 <class 'str'>
DriveThru 	 <class 'str'>
GoodForKids 	 <class 'str'>
GoodForMeal_breakfast 	 <class 'str'>
GoodFo

Assuming that for all of the above mentioned columns, missing values are indicating that particular feature is not available in the restaurant. Hence marking null values as 'false' and converting the values to 0's and 1's.

In [55]:
for x in list_cat:
    businessDF[x]=businessDF[x].fillna(False)

In [56]:
for x in list_cat:
    businessDF[x]=businessDF[x].replace(True,'1')
    businessDF[x]=businessDF[x].replace(False,'0')

In [57]:
businessDF.head(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,AgesAllowed,Alcohol,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,Ambience_upscale,BikeParking,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_lot,BusinessParking_street,BusinessParking_valet,BusinessParking_validated,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_halal,DietaryRestrictions_kosher,DietaryRestrictions_soy-free,DietaryRestrictions_vegan,DietaryRestrictions_vegetarian,DogsAllowed,DriveThru,GoodForKids,GoodForMeal_breakfast,GoodForMeal_brunch,GoodForMeal_dessert,GoodForMeal_dinner,GoodForMeal_latenight,GoodForMeal_lunch,HappyHour,HasTV,Music_no_music,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,category_Afghan,category_African,category_American (New),category_American (Traditional),category_Arabian,category_Arcades,category_Argentine,category_Armenian,category_Arts & Entertainment,category_Asian Fusion,category_Australian,category_Austrian,category_Bagels,category_Bakeries,category_Barbeque,category_Bars,category_Beer Bar,category_Beer Gardens,category_Beer,category_Belgian,category_Bistros,category_Brasseries,category_Brazilian,category_Breakfast & Brunch,category_Breweries,category_Brewpubs,category_British,category_Bubble Tea,category_Buffets,category_Burgers,category_Cafes,category_Cafeteria,category_Cajun/Creole,category_Cambodian,category_Cantonese,category_Caribbean,category_Casinos,category_Caterers,category_Cheesesteaks,category_Chicken Shop,category_Chicken Wings,category_Chinese,category_Chocolatiers & Shops,category_Cocktail Bars,category_Coffee & Tea Supplies,category_Coffee & Tea,category_Coffee Roasteries,category_Coffeeshops,category_Colombian,category_Comfort Food,category_Creperies,category_Cuban,category_Cupcakes,category_Custom Cakes,category_Czech,category_Delis,category_Desserts,category_Dim Sum,category_Diners,category_Dinner Theater,category_Do-It-Yourself Food,category_Dominican,category_Donuts,category_Eatertainment,category_Ethiopian,category_Ethnic Food,category_Event Planning & Services,category_Falafel,category_Fast Food,category_Festivals,category_Filipino,category_Fish & Chips,category_Fondue,category_Food Court,category_Food Delivery Services,category_Food Stands,category_Food Trucks,category_Food,category_French,category_Gastropubs,category_German,category_Gluten-Free,category_Greek,category_Grocery,category_Halal,category_Hawaiian,category_Herbs & Spices,category_Hookah Bars,category_Hot Dogs,category_Hot Pot,category_Hotel bar,category_Hotels,category_Ice Cream & Frozen Yogurt,category_Imported Food,category_Indian,category_Indonesian,category_Irish Pub,category_Irish,category_Italian,category_Izakaya,category_Japanese,category_Juice Bars & Smoothies,category_Kebab,category_Korean,category_Kosher,category_Laotian,category_Latin American,category_Lebanese,category_Live/Raw Food,category_Local Flavor,category_Lounges,category_Macarons,category_Malaysian,category_Marinas,category_Meat Shops,category_Mediterranean,category_Mexican,category_Middle Eastern,category_Modern European,category_Mongolian,category_Moroccan,category_Music Venues,category_New Mexican Cuisine,category_Nightlife,category_Noodles,category_Pakistani,category_Pan Asian,category_Pasta Shops,category_Patisserie/Cake Shop,category_Persian/Iranian,category_Peruvian,category_Pizza,category_Polish,category_Popcorn Shops,category_Poutineries,category_Pretzels,category_Pubs,category_Puerto Rican,category_Ramen,category_Russian,category_Salad,category_Salvadoran,category_Sandwiches,category_Scandinavian,category_Seafood,category_Soul Food,category_Soup,category_Southern,category_Spanish,category_Specialty Food,category_Sports Bars,cat

#### Cuisine Category

All of the below mentioned category columns are converted to 0's and 1's, after handing their missing values as 0's and converting their datatype to integer.

In [58]:
category_cols = ['category_Afghan','category_African', 'category_American (New)','category_American (Traditional)',
'category_Arabian','category_Arcades', 'category_Argentine', 'category_Armenian',
'category_Arts & Entertainment', 'category_Asian Fusion','category_Australian', 'category_Austrian', 'category_Bagels',
'category_Bakeries', 'category_Barbeque', 'category_Bars','category_Beer Bar', 'category_Beer Gardens', 'category_Beer',
'category_Belgian', 'category_Bistros', 'category_Brasseries',
'category_Brazilian', 'category_Breakfast & Brunch','category_Breweries', 'category_Brewpubs', 'category_British',
'category_Bubble Tea', 'category_Buffets', 'category_Burgers','category_Cafes', 'category_Cafeteria', 'category_Cajun/Creole',
'category_Cambodian', 'category_Cantonese', 'category_Caribbean',
'category_Casinos', 'category_Caterers', 'category_Cheesesteaks',
'category_Chicken Shop', 'category_Chicken Wings','category_Chinese', 'category_Chocolatiers & Shops',
'category_Cocktail Bars', 'category_Coffee & Tea Supplies','category_Coffee & Tea', 'category_Coffee Roasteries',
'category_Coffeeshops', 'category_Colombian','category_Comfort Food', 'category_Creperies', 'category_Cuban',
'category_Cupcakes', 'category_Custom Cakes', 'category_Czech',
'category_Delis', 'category_Desserts', 'category_Dim Sum',
'category_Diners', 'category_Dinner Theater','category_Do-It-Yourself Food', 'category_Dominican',
'category_Donuts', 'category_Eatertainment', 'category_Ethiopian','category_Ethnic Food', 'category_Event Planning & Services',
'category_Falafel', 'category_Fast Food', 'category_Festivals',
'category_Filipino', 'category_Fish & Chips', 'category_Fondue',
'category_Food Court', 'category_Food Delivery Services',
'category_Food Stands', 'category_Food Trucks', 'category_Food',
'category_French', 'category_Gastropubs', 'category_German',
'category_Gluten-Free', 'category_Greek', 'category_Grocery',
'category_Halal', 'category_Hawaiian', 'category_Herbs & Spices',
'category_Hookah Bars', 'category_Hot Dogs', 'category_Hot Pot',
'category_Hotel bar', 'category_Hotels',
'category_Ice Cream & Frozen Yogurt', 'category_Imported Food',
'category_Indian', 'category_Indonesian', 'category_Irish Pub',
'category_Irish', 'category_Italian', 'category_Izakaya',
'category_Japanese', 'category_Juice Bars & Smoothies',
'category_Kebab', 'category_Korean', 'category_Kosher',
'category_Laotian', 'category_Latin American', 'category_Lebanese',
'category_Live/Raw Food', 'category_Local Flavor',
'category_Lounges', 'category_Macarons', 'category_Malaysian',
'category_Marinas', 'category_Meat Shops', 'category_Mediterranean',
'category_Mexican', 'category_Middle Eastern',
'category_Modern European', 'category_Mongolian',
'category_Moroccan', 'category_Music Venues',
'category_New Mexican Cuisine', 'category_Nightlife',
'category_Noodles', 'category_Pakistani', 'category_Pan Asian',
'category_Pasta Shops', 'category_Patisserie/Cake Shop',
'category_Persian/Iranian', 'category_Peruvian', 'category_Pizza',
'category_Polish', 'category_Popcorn Shops', 'category_Poutineries',
'category_Pretzels', 'category_Pubs', 'category_Puerto Rican',
'category_Ramen', 'category_Russian','category_Salad', 'category_Salvadoran', 'category_Sandwiches',
'category_Scandinavian', 'category_Seafood', 'category_Soul Food',
'category_Soup', 'category_Southern', 'category_Spanish',
'category_Specialty Food', 'category_Sports Bars',
'category_Steakhouses', 'category_Street Vendors',
'category_Supper Clubs', 'category_Sushi Bars', 'category_Szechuan',
'category_Tacos', 'category_Taiwanese', 'category_Tapas Bars',
'category_Tapas/Small Plates', 'category_Tea Rooms',
'category_Teppanyaki', 'category_Tex-Mex', 'category_Thai',
'category_Themed Cafes', 'category_Trinidadian', 'category_Turkish',
'category_Ukrainian', 'category_Uzbek', 'category_Vegan',
'category_Vegetarian', 'category_Vietnamese', 'category_Waffles',
'category_Wine & Spirits', 'category_Wine Bars', 'category_Wraps']

In [59]:
businessDF[category_cols].head(5)

,category_Afghan,category_African,category_American (New),category_American (Traditional),category_Arabian,category_Arcades,category_Argentine,category_Armenian,category_Arts & Entertainment,category_Asian Fusion,category_Australian,category_Austrian,category_Bagels,category_Bakeries,category_Barbeque,category_Bars,category_Beer Bar,category_Beer Gardens,category_Beer,category_Belgian,category_Bistros,category_Brasseries,category_Brazilian,category_Breakfast & Brunch,category_Breweries,category_Brewpubs,category_British,category_Bubble Tea,category_Buffets,category_Burgers,category_Cafes,category_Cafeteria,category_Cajun/Creole,category_Cambodian,category_Cantonese,category_Caribbean,category_Casinos,category_Caterers,category_Cheesesteaks,category_Chicken Shop,category_Chicken Wings,category_Chinese,category_Chocolatiers & Shops,category_Cocktail Bars,category_Coffee & Tea Supplies,category_Coffee & Tea,category_Coffee Roasteries,category_Coffeeshops,category_Colombian,category_Comfort Food,category_Creperies,category_Cuban,category_Cupcakes,category_Custom Cakes,category_Czech,category_Delis,category_Desserts,category_Dim Sum,category_Diners,category_Dinner Theater,category_Do-It-Yourself Food,category_Dominican,category_Donuts,category_Eatertainment,category_Ethiopian,category_Ethnic Food,category_Event Planning & Services,category_Falafel,category_Fast Food,category_Festivals,category_Filipino,category_Fish & Chips,category_Fondue,category_Food Court,category_Food Delivery Services,category_Food Stands,category_Food Trucks,category_Food,category_French,category_Gastropubs,category_German,category_Gluten-Free,category_Greek,category_Grocery,category_Halal,category_Hawaiian,category_Herbs & Spices,category_Hookah Bars,category_Hot Dogs,category_Hot Pot,category_Hotel bar,category_Hotels,category_Ice Cream & Frozen Yogurt,category_Imported Food,category_Indian,category_Indonesian,category_Irish Pub,category_Irish,category_Italian,category_Izakaya,category_Japanese,category_Juice Bars & Smoothies,category_Kebab,category_Korean,category_Kosher,category_Laotian,category_Latin American,category_Lebanese,category_Live/Raw Food,category_Local Flavor,category_Lounges,category_Macarons,category_Malaysian,category_Marinas,category_Meat Shops,category_Mediterranean,category_Mexican,category_Middle Eastern,category_Modern European,category_Mongolian,category_Moroccan,category_Music Venues,category_New Mexican Cuisine,category_Nightlife,category_Noodles,category_Pakistani,category_Pan Asian,category_Pasta Shops,category_Patisserie/Cake Shop,category_Persian/Iranian,category_Peruvian,category_Pizza,category_Polish,category_Popcorn Shops,category_Poutineries,category_Pretzels,category_Pubs,category_Puerto Rican,category_Ramen,category_Russian,category_Salad,category_Salvadoran,category_Sandwiches,category_Scandinavian,category_Seafood,category_Soul Food,category_Soup,category_Southern,category_Spanish,category_Specialty Food,category_Sports Bars,category_Steakhouses,category_Street Vendors,category_Supper Clubs,category_Sushi Bars,category_Szechuan,category_Tacos,category_Taiwanese,category_Tapas Bars,category_Tapas/Small Plates,category_Tea Rooms,category_Teppanyaki,category_Tex-Mex,category_Thai,category_Themed Cafes,category_Trinidadian,category_Turkish,category_Ukrainian,category_Uzbek,category_Vegan,category_Vegetarian,category_Vietnamese,category_Waffles,category_Wine & Spirits,category_Wine Bars,category_Wraps
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [60]:
len(category_cols)

176

There is a total of 176 category columns in our restaurant business dataframe.

In [61]:
for x in category_cols:
    businessDF[x]=businessDF[x].fillna(0.0)

In [62]:
businessDF[category_cols].head(5)

,category_Afghan,category_African,category_American (New),category_American (Traditional),category_Arabian,category_Arcades,category_Argentine,category_Armenian,category_Arts & Entertainment,category_Asian Fusion,category_Australian,category_Austrian,category_Bagels,category_Bakeries,category_Barbeque,category_Bars,category_Beer Bar,category_Beer Gardens,category_Beer,category_Belgian,category_Bistros,category_Brasseries,category_Brazilian,category_Breakfast & Brunch,category_Breweries,category_Brewpubs,category_British,category_Bubble Tea,category_Buffets,category_Burgers,category_Cafes,category_Cafeteria,category_Cajun/Creole,category_Cambodian,category_Cantonese,category_Caribbean,category_Casinos,category_Caterers,category_Cheesesteaks,category_Chicken Shop,category_Chicken Wings,category_Chinese,category_Chocolatiers & Shops,category_Cocktail Bars,category_Coffee & Tea Supplies,category_Coffee & Tea,category_Coffee Roasteries,category_Coffeeshops,category_Colombian,category_Comfort Food,category_Creperies,category_Cuban,category_Cupcakes,category_Custom Cakes,category_Czech,category_Delis,category_Desserts,category_Dim Sum,category_Diners,category_Dinner Theater,category_Do-It-Yourself Food,category_Dominican,category_Donuts,category_Eatertainment,category_Ethiopian,category_Ethnic Food,category_Event Planning & Services,category_Falafel,category_Fast Food,category_Festivals,category_Filipino,category_Fish & Chips,category_Fondue,category_Food Court,category_Food Delivery Services,category_Food Stands,category_Food Trucks,category_Food,category_French,category_Gastropubs,category_German,category_Gluten-Free,category_Greek,category_Grocery,category_Halal,category_Hawaiian,category_Herbs & Spices,category_Hookah Bars,category_Hot Dogs,category_Hot Pot,category_Hotel bar,category_Hotels,category_Ice Cream & Frozen Yogurt,category_Imported Food,category_Indian,category_Indonesian,category_Irish Pub,category_Irish,category_Italian,category_Izakaya,category_Japanese,category_Juice Bars & Smoothies,category_Kebab,category_Korean,category_Kosher,category_Laotian,category_Latin American,category_Lebanese,category_Live/Raw Food,category_Local Flavor,category_Lounges,category_Macarons,category_Malaysian,category_Marinas,category_Meat Shops,category_Mediterranean,category_Mexican,category_Middle Eastern,category_Modern European,category_Mongolian,category_Moroccan,category_Music Venues,category_New Mexican Cuisine,category_Nightlife,category_Noodles,category_Pakistani,category_Pan Asian,category_Pasta Shops,category_Patisserie/Cake Shop,category_Persian/Iranian,category_Peruvian,category_Pizza,category_Polish,category_Popcorn Shops,category_Poutineries,category_Pretzels,category_Pubs,category_Puerto Rican,category_Ramen,category_Russian,category_Salad,category_Salvadoran,category_Sandwiches,category_Scandinavian,category_Seafood,category_Soul Food,category_Soup,category_Southern,category_Spanish,category_Specialty Food,category_Sports Bars,category_Steakhouses,category_Street Vendors,category_Supper Clubs,category_Sushi Bars,category_Szechuan,category_Tacos,category_Taiwanese,category_Tapas Bars,category_Tapas/Small Plates,category_Tea Rooms,category_Teppanyaki,category_Tex-Mex,category_Thai,category_Themed Cafes,category_Trinidadian,category_Turkish,category_Ukrainian,category_Uzbek,category_Vegan,category_Vegetarian,category_Vietnamese,category_Waffles,category_Wine & Spirits,category_Wine Bars,category_Wraps
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [63]:
import string

In [64]:
for x in category_cols:
    businessDF[x]=businessDF[x].astype(int)

In [65]:
businessDF[category_cols].head(4)

,category_Afghan,category_African,category_American (New),category_American (Traditional),category_Arabian,category_Arcades,category_Argentine,category_Armenian,category_Arts & Entertainment,category_Asian Fusion,category_Australian,category_Austrian,category_Bagels,category_Bakeries,category_Barbeque,category_Bars,category_Beer Bar,category_Beer Gardens,category_Beer,category_Belgian,category_Bistros,category_Brasseries,category_Brazilian,category_Breakfast & Brunch,category_Breweries,category_Brewpubs,category_British,category_Bubble Tea,category_Buffets,category_Burgers,category_Cafes,category_Cafeteria,category_Cajun/Creole,category_Cambodian,category_Cantonese,category_Caribbean,category_Casinos,category_Caterers,category_Cheesesteaks,category_Chicken Shop,category_Chicken Wings,category_Chinese,category_Chocolatiers & Shops,category_Cocktail Bars,category_Coffee & Tea Supplies,category_Coffee & Tea,category_Coffee Roasteries,category_Coffeeshops,category_Colombian,category_Comfort Food,category_Creperies,category_Cuban,category_Cupcakes,category_Custom Cakes,category_Czech,category_Delis,category_Desserts,category_Dim Sum,category_Diners,category_Dinner Theater,category_Do-It-Yourself Food,category_Dominican,category_Donuts,category_Eatertainment,category_Ethiopian,category_Ethnic Food,category_Event Planning & Services,category_Falafel,category_Fast Food,category_Festivals,category_Filipino,category_Fish & Chips,category_Fondue,category_Food Court,category_Food Delivery Services,category_Food Stands,category_Food Trucks,category_Food,category_French,category_Gastropubs,category_German,category_Gluten-Free,category_Greek,category_Grocery,category_Halal,category_Hawaiian,category_Herbs & Spices,category_Hookah Bars,category_Hot Dogs,category_Hot Pot,category_Hotel bar,category_Hotels,category_Ice Cream & Frozen Yogurt,category_Imported Food,category_Indian,category_Indonesian,category_Irish Pub,category_Irish,category_Italian,category_Izakaya,category_Japanese,category_Juice Bars & Smoothies,category_Kebab,category_Korean,category_Kosher,category_Laotian,category_Latin American,category_Lebanese,category_Live/Raw Food,category_Local Flavor,category_Lounges,category_Macarons,category_Malaysian,category_Marinas,category_Meat Shops,category_Mediterranean,category_Mexican,category_Middle Eastern,category_Modern European,category_Mongolian,category_Moroccan,category_Music Venues,category_New Mexican Cuisine,category_Nightlife,category_Noodles,category_Pakistani,category_Pan Asian,category_Pasta Shops,category_Patisserie/Cake Shop,category_Persian/Iranian,category_Peruvian,category_Pizza,category_Polish,category_Popcorn Shops,category_Poutineries,category_Pretzels,category_Pubs,category_Puerto Rican,category_Ramen,category_Russian,category_Salad,category_Salvadoran,category_Sandwiches,category_Scandinavian,category_Seafood,category_Soul Food,category_Soup,category_Southern,category_Spanish,category_Specialty Food,category_Sports Bars,category_Steakhouses,category_Street Vendors,category_Supper Clubs,category_Sushi Bars,category_Szechuan,category_Tacos,category_Taiwanese,category_Tapas Bars,category_Tapas/Small Plates,category_Tea Rooms,category_Teppanyaki,category_Tex-Mex,category_Thai,category_Themed Cafes,category_Trinidadian,category_Turkish,category_Ukrainian,category_Uzbek,category_Vegan,category_Vegetarian,category_Vietnamese,category_Waffles,category_Wine & Spirits,category_Wine Bars,category_Wraps
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Re-running the first loop, to determine if any columns' missing values were not handled.

In [66]:
list_nan=[]

In [67]:
for colmn in businessDF.columns.values:
    print('Column name: ',colmn)
    print('No of Unique values: ',businessDF[colmn].nunique())
    print('Unique values: ',businessDF[colmn].unique())
    print('NaN count: ',businessDF[colmn].isnull().sum())
    if(colmn not in ['business_id','name','neighborhood','address','city','state','postal_code','latitude','longitude','stars',
                     'review_count']):
        print('Attribute counts:\n',businessDF[colmn].value_counts(),'\n')
        print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    else:
        print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    total_nan=businessDF[colmn].isnull().sum()
    if(total_nan > 0):
        list_nan.append(colmn)

Column name:  business_id
No of Unique values:  7367
Unique values:  ['rDMptJYWtnMhpQu_rRXHng' '1WBkAuQg81kokZIPMpn9Zg'
 'iPa__LOhse-hobC2Xmp-Kw' ... '5zva2MTtB5IX6TaoVLL-NA'
 'Gr-2oBg4XyduSKbvnE-i9g' 'UdEmYOnk2iJDY9lpEPAlJQ']
NaN count:  0
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  name
No of Unique values:  4172
Unique values:  ["McDonald's" 'Charr An American Burger Bar' 'Little Caesars Pizza' ...
 "Yin's Chinese Resturant" 'Zorbas Grill'
 'Salt & Lime Modern Mexican Grill']
NaN count:  0
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  address
No of Unique values:  6721
Unique values:  ['719 E Thunderbird Rd' '777 E Thunderbird Rd, Ste 107'
 '1635 E Camelback Rd' ... '1950 W Indian School Rd'
 '9397 E Shea Blvd, Ste 115' '590 N Alma School Rd, Ste 35']
NaN count:  86
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  city
No of Unique values:  41
U

Column name:  BikeParking
No of Unique values:  2
Unique values:  ['1' '0']
NaN count:  0
Attribute counts:
 1    5444
0    1923
Name: BikeParking, dtype: int64 

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  BusinessAcceptsCreditCards
No of Unique values:  2
Unique values:  ['1' '0']
NaN count:  0
Attribute counts:
 1    7097
0     270
Name: BusinessAcceptsCreditCards, dtype: int64 

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  BusinessParking_garage
No of Unique values:  2
Unique values:  ['0' '1']
NaN count:  0
Attribute counts:
 0    7139
1     228
Name: BusinessParking_garage, dtype: int64 

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  BusinessParking_lot
No of Unique values:  2
Unique values:  ['1' '0']
NaN count:  0
Attribute counts:
 1    3777
0    3590
Name: BusinessParking_lot, dtype: int64 

+++++++++++++++++++++++++++++++++++++++++++

Unique values:  [0 1]
NaN count:  0
Attribute counts:
 0    7349
1      18
Name: category_Coffee Roasteries, dtype: int64 

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  category_Coffeeshops
No of Unique values:  2
Unique values:  [0 1]
NaN count:  0
Attribute counts:
 0    7366
1       1
Name: category_Coffeeshops, dtype: int64 

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  category_Colombian
No of Unique values:  2
Unique values:  [0 1]
NaN count:  0
Attribute counts:
 0    7364
1       3
Name: category_Colombian, dtype: int64 

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  category_Comfort Food
No of Unique values:  2
Unique values:  [0 1]
NaN count:  0
Attribute counts:
 0    7320
1      47
Name: category_Comfort Food, dtype: int64 

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  category_Creperie

Unique values:  [0 1]
NaN count:  0
Attribute counts:
 0    7360
1       7
Name: category_Spanish, dtype: int64 

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  category_Specialty Food
No of Unique values:  2
Unique values:  [0 1]
NaN count:  0
Attribute counts:
 0    7206
1     161
Name: category_Specialty Food, dtype: int64 

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  category_Sports Bars
No of Unique values:  2
Unique values:  [0 1]
NaN count:  0
Attribute counts:
 0    7089
1     278
Name: category_Sports Bars, dtype: int64 

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  category_Steakhouses
No of Unique values:  2
Unique values:  [0 1]
NaN count:  0
Attribute counts:
 0    7188
1     179
Name: category_Steakhouses, dtype: int64 

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  category_Street Ven

In [68]:
len(list_nan)

2

In [69]:
list_nan

['address', 'postal_code']

We can see that, 2 columns namely 'address' and 'postal_code' are not having their missing values handled. Also, since we are just recommending the restaurants to the user we are not going to handle 'address' columns' missing values. 
The 'postal_code' columns' values are being handled after plotting the map in Tableau for EDA.

#### postal code

In [70]:
businessDF[businessDF['postal_code'].isnull()]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,AgesAllowed,Alcohol,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,Ambience_upscale,BikeParking,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_lot,BusinessParking_street,BusinessParking_valet,BusinessParking_validated,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_halal,DietaryRestrictions_kosher,DietaryRestrictions_soy-free,DietaryRestrictions_vegan,DietaryRestrictions_vegetarian,DogsAllowed,DriveThru,GoodForKids,GoodForMeal_breakfast,GoodForMeal_brunch,GoodForMeal_dessert,GoodForMeal_dinner,GoodForMeal_latenight,GoodForMeal_lunch,HappyHour,HasTV,Music_no_music,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,category_Afghan,category_African,category_American (New),category_American (Traditional),category_Arabian,category_Arcades,category_Argentine,category_Armenian,category_Arts & Entertainment,category_Asian Fusion,category_Australian,category_Austrian,category_Bagels,category_Bakeries,category_Barbeque,category_Bars,category_Beer Bar,category_Beer Gardens,category_Beer,category_Belgian,category_Bistros,category_Brasseries,category_Brazilian,category_Breakfast & Brunch,category_Breweries,category_Brewpubs,category_British,category_Bubble Tea,category_Buffets,category_Burgers,category_Cafes,category_Cafeteria,category_Cajun/Creole,category_Cambodian,category_Cantonese,category_Caribbean,category_Casinos,category_Caterers,category_Cheesesteaks,category_Chicken Shop,category_Chicken Wings,category_Chinese,category_Chocolatiers & Shops,category_Cocktail Bars,category_Coffee & Tea Supplies,category_Coffee & Tea,category_Coffee Roasteries,category_Coffeeshops,category_Colombian,category_Comfort Food,category_Creperies,category_Cuban,category_Cupcakes,category_Custom Cakes,category_Czech,category_Delis,category_Desserts,category_Dim Sum,category_Diners,category_Dinner Theater,category_Do-It-Yourself Food,category_Dominican,category_Donuts,category_Eatertainment,category_Ethiopian,category_Ethnic Food,category_Event Planning & Services,category_Falafel,category_Fast Food,category_Festivals,category_Filipino,category_Fish & Chips,category_Fondue,category_Food Court,category_Food Delivery Services,category_Food Stands,category_Food Trucks,category_Food,category_French,category_Gastropubs,category_German,category_Gluten-Free,category_Greek,category_Grocery,category_Halal,category_Hawaiian,category_Herbs & Spices,category_Hookah Bars,category_Hot Dogs,category_Hot Pot,category_Hotel bar,category_Hotels,category_Ice Cream & Frozen Yogurt,category_Imported Food,category_Indian,category_Indonesian,category_Irish Pub,category_Irish,category_Italian,category_Izakaya,category_Japanese,category_Juice Bars & Smoothies,category_Kebab,category_Korean,category_Kosher,category_Laotian,category_Latin American,category_Lebanese,category_Live/Raw Food,category_Local Flavor,category_Lounges,category_Macarons,category_Malaysian,category_Marinas,category_Meat Shops,category_Mediterranean,category_Mexican,category_Middle Eastern,category_Modern European,category_Mongolian,category_Moroccan,category_Music Venues,category_New Mexican Cuisine,category_Nightlife,category_Noodles,category_Pakistani,category_Pan Asian,category_Pasta Shops,category_Patisserie/Cake Shop,category_Persian/Iranian,category_Peruvian,category_Pizza,category_Polish,category_Popcorn Shops,category_Poutineries,category_Pretzels,category_Pubs,category_Puerto Rican,category_Ramen,category_Russian,category_Salad,category_Salvadoran,category_Sandwiches,category_Scandinavian,category_Seafood,category_Soul Food,category_Soup,category_Southern,category_Spanish,category_Specialty Food,category_Sports Bars,cat

In [71]:
businessDF['postal_code'].isnull().sum()

9

There is a total of 9 businesses' that are not having zip code values. These values are determined from Tableau afer plotting the map and taking the closest value to the missing zip code value or determing the actual zip code value.
They are present in the zipFill vector below. From the above python statement we are determing the bussiness names for handling their missing zip codes as shown below.

In [72]:
zipFill=[85004.0,85210.0,85374.0,85003.0,85331.0,85206.0,85003.0,85003.0,85027.0]

j=0
for x in ['Paradise Valley Burger Chase Field','Hot Cookie Truck','Vogue Bistro','Taquizas "Yiyo" Catering and Party Supplies',
          'Wandering Donkey','Zita Mediterranean Eatery','Leafy Sea Dragon','Satay Hut','Time Square']:
    businessDF.iloc[businessDF[businessDF['name']==x]['postal_code'].index[0],5]= zipFill[j]
    j=j+1                               

In [73]:
businessDF.iloc[businessDF[businessDF['business_id']=='8N5A5jW8sTG7ozGmZ_uj0Q']['postal_code'].index[0],5] = 85004.0

Verifying the above zip code changes for a particular business above.

In [74]:
businessDF[businessDF['business_id']=='8N5A5jW8sTG7ozGmZ_uj0Q']

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,AgesAllowed,Alcohol,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,Ambience_upscale,BikeParking,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_lot,BusinessParking_street,BusinessParking_valet,BusinessParking_validated,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_halal,DietaryRestrictions_kosher,DietaryRestrictions_soy-free,DietaryRestrictions_vegan,DietaryRestrictions_vegetarian,DogsAllowed,DriveThru,GoodForKids,GoodForMeal_breakfast,GoodForMeal_brunch,GoodForMeal_dessert,GoodForMeal_dinner,GoodForMeal_latenight,GoodForMeal_lunch,HappyHour,HasTV,Music_no_music,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,category_Afghan,category_African,category_American (New),category_American (Traditional),category_Arabian,category_Arcades,category_Argentine,category_Armenian,category_Arts & Entertainment,category_Asian Fusion,category_Australian,category_Austrian,category_Bagels,category_Bakeries,category_Barbeque,category_Bars,category_Beer Bar,category_Beer Gardens,category_Beer,category_Belgian,category_Bistros,category_Brasseries,category_Brazilian,category_Breakfast & Brunch,category_Breweries,category_Brewpubs,category_British,category_Bubble Tea,category_Buffets,category_Burgers,category_Cafes,category_Cafeteria,category_Cajun/Creole,category_Cambodian,category_Cantonese,category_Caribbean,category_Casinos,category_Caterers,category_Cheesesteaks,category_Chicken Shop,category_Chicken Wings,category_Chinese,category_Chocolatiers & Shops,category_Cocktail Bars,category_Coffee & Tea Supplies,category_Coffee & Tea,category_Coffee Roasteries,category_Coffeeshops,category_Colombian,category_Comfort Food,category_Creperies,category_Cuban,category_Cupcakes,category_Custom Cakes,category_Czech,category_Delis,category_Desserts,category_Dim Sum,category_Diners,category_Dinner Theater,category_Do-It-Yourself Food,category_Dominican,category_Donuts,category_Eatertainment,category_Ethiopian,category_Ethnic Food,category_Event Planning & Services,category_Falafel,category_Fast Food,category_Festivals,category_Filipino,category_Fish & Chips,category_Fondue,category_Food Court,category_Food Delivery Services,category_Food Stands,category_Food Trucks,category_Food,category_French,category_Gastropubs,category_German,category_Gluten-Free,category_Greek,category_Grocery,category_Halal,category_Hawaiian,category_Herbs & Spices,category_Hookah Bars,category_Hot Dogs,category_Hot Pot,category_Hotel bar,category_Hotels,category_Ice Cream & Frozen Yogurt,category_Imported Food,category_Indian,category_Indonesian,category_Irish Pub,category_Irish,category_Italian,category_Izakaya,category_Japanese,category_Juice Bars & Smoothies,category_Kebab,category_Korean,category_Kosher,category_Laotian,category_Latin American,category_Lebanese,category_Live/Raw Food,category_Local Flavor,category_Lounges,category_Macarons,category_Malaysian,category_Marinas,category_Meat Shops,category_Mediterranean,category_Mexican,category_Middle Eastern,category_Modern European,category_Mongolian,category_Moroccan,category_Music Venues,category_New Mexican Cuisine,category_Nightlife,category_Noodles,category_Pakistani,category_Pan Asian,category_Pasta Shops,category_Patisserie/Cake Shop,category_Persian/Iranian,category_Peruvian,category_Pizza,category_Polish,category_Popcorn Shops,category_Poutineries,category_Pretzels,category_Pubs,category_Puerto Rican,category_Ramen,category_Russian,category_Salad,category_Salvadoran,category_Sandwiches,category_Scandinavian,category_Seafood,category_Soul Food,category_Soup,category_Southern,category_Spanish,category_Specialty Food,category_Sports Bars,cat

In [75]:
businessDF[businessDF['postal_code'].isnull()]['business_id']

Series([], Name: business_id, dtype: object)

In [76]:
businessDF.iloc[businessDF[businessDF['business_id']=='o-cqIwS2nWJGWA6nNPQ2_w']['postal_code'].index[0],5] = 85210
businessDF.iloc[businessDF[businessDF['business_id']=='3LWTG3f52gdVZILl42FCAg']['postal_code'].index[0],5] = 85374
businessDF.iloc[businessDF[businessDF['business_id']=='DscqZ5DUZSWTggbv6N_j3Q']['postal_code'].index[0],5] = 85003
businessDF.iloc[businessDF[businessDF['business_id']=='BxRsg5YTYnxXEhoGcabydg']['postal_code'].index[0],5] = 85331
businessDF.iloc[businessDF[businessDF['business_id']=='R0gfUYIiC5MlLxGjRcYifw']['postal_code'].index[0],5] = 85206
businessDF.iloc[businessDF[businessDF['business_id']=='YH9jx2vlDBD47qTBoCvkoA']['postal_code'].index[0],5] = 85003
businessDF.iloc[businessDF[businessDF['business_id']=='wFJFBZDMcV0hBMYOx88IYQ']['postal_code'].index[0],5] = 85003
businessDF.iloc[businessDF[businessDF['business_id']=='tY_Hmm0rT4sRDL-geEYQmg']['postal_code'].index[0],5] = 85027

In [77]:
businessDF[businessDF['postal_code'].isnull()]['name'].count()

0

Converting 'postal_code' column values of type int, as now it contains no missing values.

In [78]:
businessDF['postal_code'] = businessDF['postal_code'].astype(int)

In [79]:
businessDF['postal_code'].dtype

dtype('int32')

#### Other  data cleansing

The below mentioned handling of business dataframe's values is due to discrepancies found in Tableau during plotting of charts, maps and graphs.
We were able to find restaurants with same name but with extra symbols, wrong city details based on the postal code provided in the data and also the business having wrong latitude and longitude values.
Such discrepancies are handled below.

In [80]:
businessDF[businessDF['name']=='Little Caesars Pizza'].head(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,AgesAllowed,Alcohol,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,Ambience_upscale,BikeParking,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_lot,BusinessParking_street,BusinessParking_valet,BusinessParking_validated,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_halal,DietaryRestrictions_kosher,DietaryRestrictions_soy-free,DietaryRestrictions_vegan,DietaryRestrictions_vegetarian,DogsAllowed,DriveThru,GoodForKids,GoodForMeal_breakfast,GoodForMeal_brunch,GoodForMeal_dessert,GoodForMeal_dinner,GoodForMeal_latenight,GoodForMeal_lunch,HappyHour,HasTV,Music_no_music,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,category_Afghan,category_African,category_American (New),category_American (Traditional),category_Arabian,category_Arcades,category_Argentine,category_Armenian,category_Arts & Entertainment,category_Asian Fusion,category_Australian,category_Austrian,category_Bagels,category_Bakeries,category_Barbeque,category_Bars,category_Beer Bar,category_Beer Gardens,category_Beer,category_Belgian,category_Bistros,category_Brasseries,category_Brazilian,category_Breakfast & Brunch,category_Breweries,category_Brewpubs,category_British,category_Bubble Tea,category_Buffets,category_Burgers,category_Cafes,category_Cafeteria,category_Cajun/Creole,category_Cambodian,category_Cantonese,category_Caribbean,category_Casinos,category_Caterers,category_Cheesesteaks,category_Chicken Shop,category_Chicken Wings,category_Chinese,category_Chocolatiers & Shops,category_Cocktail Bars,category_Coffee & Tea Supplies,category_Coffee & Tea,category_Coffee Roasteries,category_Coffeeshops,category_Colombian,category_Comfort Food,category_Creperies,category_Cuban,category_Cupcakes,category_Custom Cakes,category_Czech,category_Delis,category_Desserts,category_Dim Sum,category_Diners,category_Dinner Theater,category_Do-It-Yourself Food,category_Dominican,category_Donuts,category_Eatertainment,category_Ethiopian,category_Ethnic Food,category_Event Planning & Services,category_Falafel,category_Fast Food,category_Festivals,category_Filipino,category_Fish & Chips,category_Fondue,category_Food Court,category_Food Delivery Services,category_Food Stands,category_Food Trucks,category_Food,category_French,category_Gastropubs,category_German,category_Gluten-Free,category_Greek,category_Grocery,category_Halal,category_Hawaiian,category_Herbs & Spices,category_Hookah Bars,category_Hot Dogs,category_Hot Pot,category_Hotel bar,category_Hotels,category_Ice Cream & Frozen Yogurt,category_Imported Food,category_Indian,category_Indonesian,category_Irish Pub,category_Irish,category_Italian,category_Izakaya,category_Japanese,category_Juice Bars & Smoothies,category_Kebab,category_Korean,category_Kosher,category_Laotian,category_Latin American,category_Lebanese,category_Live/Raw Food,category_Local Flavor,category_Lounges,category_Macarons,category_Malaysian,category_Marinas,category_Meat Shops,category_Mediterranean,category_Mexican,category_Middle Eastern,category_Modern European,category_Mongolian,category_Moroccan,category_Music Venues,category_New Mexican Cuisine,category_Nightlife,category_Noodles,category_Pakistani,category_Pan Asian,category_Pasta Shops,category_Patisserie/Cake Shop,category_Persian/Iranian,category_Peruvian,category_Pizza,category_Polish,category_Popcorn Shops,category_Poutineries,category_Pretzels,category_Pubs,category_Puerto Rican,category_Ramen,category_Russian,category_Salad,category_Salvadoran,category_Sandwiches,category_Scandinavian,category_Seafood,category_Soul Food,category_Soup,category_Southern,category_Spanish,category_Specialty Food,category_Sports Bars,cat

In [81]:
businessDF.loc[businessDF['business_id']=='ivYI5FZ7ULlfaqsc-wIVSQ']

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,AgesAllowed,Alcohol,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,Ambience_upscale,BikeParking,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_lot,BusinessParking_street,BusinessParking_valet,BusinessParking_validated,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_halal,DietaryRestrictions_kosher,DietaryRestrictions_soy-free,DietaryRestrictions_vegan,DietaryRestrictions_vegetarian,DogsAllowed,DriveThru,GoodForKids,GoodForMeal_breakfast,GoodForMeal_brunch,GoodForMeal_dessert,GoodForMeal_dinner,GoodForMeal_latenight,GoodForMeal_lunch,HappyHour,HasTV,Music_no_music,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,category_Afghan,category_African,category_American (New),category_American (Traditional),category_Arabian,category_Arcades,category_Argentine,category_Armenian,category_Arts & Entertainment,category_Asian Fusion,category_Australian,category_Austrian,category_Bagels,category_Bakeries,category_Barbeque,category_Bars,category_Beer Bar,category_Beer Gardens,category_Beer,category_Belgian,category_Bistros,category_Brasseries,category_Brazilian,category_Breakfast & Brunch,category_Breweries,category_Brewpubs,category_British,category_Bubble Tea,category_Buffets,category_Burgers,category_Cafes,category_Cafeteria,category_Cajun/Creole,category_Cambodian,category_Cantonese,category_Caribbean,category_Casinos,category_Caterers,category_Cheesesteaks,category_Chicken Shop,category_Chicken Wings,category_Chinese,category_Chocolatiers & Shops,category_Cocktail Bars,category_Coffee & Tea Supplies,category_Coffee & Tea,category_Coffee Roasteries,category_Coffeeshops,category_Colombian,category_Comfort Food,category_Creperies,category_Cuban,category_Cupcakes,category_Custom Cakes,category_Czech,category_Delis,category_Desserts,category_Dim Sum,category_Diners,category_Dinner Theater,category_Do-It-Yourself Food,category_Dominican,category_Donuts,category_Eatertainment,category_Ethiopian,category_Ethnic Food,category_Event Planning & Services,category_Falafel,category_Fast Food,category_Festivals,category_Filipino,category_Fish & Chips,category_Fondue,category_Food Court,category_Food Delivery Services,category_Food Stands,category_Food Trucks,category_Food,category_French,category_Gastropubs,category_German,category_Gluten-Free,category_Greek,category_Grocery,category_Halal,category_Hawaiian,category_Herbs & Spices,category_Hookah Bars,category_Hot Dogs,category_Hot Pot,category_Hotel bar,category_Hotels,category_Ice Cream & Frozen Yogurt,category_Imported Food,category_Indian,category_Indonesian,category_Irish Pub,category_Irish,category_Italian,category_Izakaya,category_Japanese,category_Juice Bars & Smoothies,category_Kebab,category_Korean,category_Kosher,category_Laotian,category_Latin American,category_Lebanese,category_Live/Raw Food,category_Local Flavor,category_Lounges,category_Macarons,category_Malaysian,category_Marinas,category_Meat Shops,category_Mediterranean,category_Mexican,category_Middle Eastern,category_Modern European,category_Mongolian,category_Moroccan,category_Music Venues,category_New Mexican Cuisine,category_Nightlife,category_Noodles,category_Pakistani,category_Pan Asian,category_Pasta Shops,category_Patisserie/Cake Shop,category_Persian/Iranian,category_Peruvian,category_Pizza,category_Polish,category_Popcorn Shops,category_Poutineries,category_Pretzels,category_Pubs,category_Puerto Rican,category_Ramen,category_Russian,category_Salad,category_Salvadoran,category_Sandwiches,category_Scandinavian,category_Seafood,category_Soul Food,category_Soup,category_Southern,category_Spanish,category_Specialty Food,category_Sports Bars,cat

In [82]:
businessDF[businessDF['business_id']=='ivYI5FZ7ULlfaqsc-wIVSQ']

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,AgesAllowed,Alcohol,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,Ambience_upscale,BikeParking,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_lot,BusinessParking_street,BusinessParking_valet,BusinessParking_validated,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_halal,DietaryRestrictions_kosher,DietaryRestrictions_soy-free,DietaryRestrictions_vegan,DietaryRestrictions_vegetarian,DogsAllowed,DriveThru,GoodForKids,GoodForMeal_breakfast,GoodForMeal_brunch,GoodForMeal_dessert,GoodForMeal_dinner,GoodForMeal_latenight,GoodForMeal_lunch,HappyHour,HasTV,Music_no_music,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,category_Afghan,category_African,category_American (New),category_American (Traditional),category_Arabian,category_Arcades,category_Argentine,category_Armenian,category_Arts & Entertainment,category_Asian Fusion,category_Australian,category_Austrian,category_Bagels,category_Bakeries,category_Barbeque,category_Bars,category_Beer Bar,category_Beer Gardens,category_Beer,category_Belgian,category_Bistros,category_Brasseries,category_Brazilian,category_Breakfast & Brunch,category_Breweries,category_Brewpubs,category_British,category_Bubble Tea,category_Buffets,category_Burgers,category_Cafes,category_Cafeteria,category_Cajun/Creole,category_Cambodian,category_Cantonese,category_Caribbean,category_Casinos,category_Caterers,category_Cheesesteaks,category_Chicken Shop,category_Chicken Wings,category_Chinese,category_Chocolatiers & Shops,category_Cocktail Bars,category_Coffee & Tea Supplies,category_Coffee & Tea,category_Coffee Roasteries,category_Coffeeshops,category_Colombian,category_Comfort Food,category_Creperies,category_Cuban,category_Cupcakes,category_Custom Cakes,category_Czech,category_Delis,category_Desserts,category_Dim Sum,category_Diners,category_Dinner Theater,category_Do-It-Yourself Food,category_Dominican,category_Donuts,category_Eatertainment,category_Ethiopian,category_Ethnic Food,category_Event Planning & Services,category_Falafel,category_Fast Food,category_Festivals,category_Filipino,category_Fish & Chips,category_Fondue,category_Food Court,category_Food Delivery Services,category_Food Stands,category_Food Trucks,category_Food,category_French,category_Gastropubs,category_German,category_Gluten-Free,category_Greek,category_Grocery,category_Halal,category_Hawaiian,category_Herbs & Spices,category_Hookah Bars,category_Hot Dogs,category_Hot Pot,category_Hotel bar,category_Hotels,category_Ice Cream & Frozen Yogurt,category_Imported Food,category_Indian,category_Indonesian,category_Irish Pub,category_Irish,category_Italian,category_Izakaya,category_Japanese,category_Juice Bars & Smoothies,category_Kebab,category_Korean,category_Kosher,category_Laotian,category_Latin American,category_Lebanese,category_Live/Raw Food,category_Local Flavor,category_Lounges,category_Macarons,category_Malaysian,category_Marinas,category_Meat Shops,category_Mediterranean,category_Mexican,category_Middle Eastern,category_Modern European,category_Mongolian,category_Moroccan,category_Music Venues,category_New Mexican Cuisine,category_Nightlife,category_Noodles,category_Pakistani,category_Pan Asian,category_Pasta Shops,category_Patisserie/Cake Shop,category_Persian/Iranian,category_Peruvian,category_Pizza,category_Polish,category_Popcorn Shops,category_Poutineries,category_Pretzels,category_Pubs,category_Puerto Rican,category_Ramen,category_Russian,category_Salad,category_Salvadoran,category_Sandwiches,category_Scandinavian,category_Seafood,category_Soul Food,category_Soup,category_Southern,category_Spanish,category_Specialty Food,category_Sports Bars,cat

In [83]:
businessDF.iloc[619,3]='Phoenix'

In [84]:
businessDF[businessDF['business_id']=='ivYI5FZ7ULlfaqsc-wIVSQ']['city']

619    Phoenix
Name: city, dtype: object

In [85]:
businessDF.iloc[businessDF[businessDF['business_id']=='j8j0IV_eemJpFuRRvdNobQ']['city'].index[0],3] = 'Phoenix'

In [86]:
businessDF[businessDF['business_id']=='j8j0IV_eemJpFuRRvdNobQ']

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,AgesAllowed,Alcohol,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,Ambience_upscale,BikeParking,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_lot,BusinessParking_street,BusinessParking_valet,BusinessParking_validated,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_halal,DietaryRestrictions_kosher,DietaryRestrictions_soy-free,DietaryRestrictions_vegan,DietaryRestrictions_vegetarian,DogsAllowed,DriveThru,GoodForKids,GoodForMeal_breakfast,GoodForMeal_brunch,GoodForMeal_dessert,GoodForMeal_dinner,GoodForMeal_latenight,GoodForMeal_lunch,HappyHour,HasTV,Music_no_music,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,category_Afghan,category_African,category_American (New),category_American (Traditional),category_Arabian,category_Arcades,category_Argentine,category_Armenian,category_Arts & Entertainment,category_Asian Fusion,category_Australian,category_Austrian,category_Bagels,category_Bakeries,category_Barbeque,category_Bars,category_Beer Bar,category_Beer Gardens,category_Beer,category_Belgian,category_Bistros,category_Brasseries,category_Brazilian,category_Breakfast & Brunch,category_Breweries,category_Brewpubs,category_British,category_Bubble Tea,category_Buffets,category_Burgers,category_Cafes,category_Cafeteria,category_Cajun/Creole,category_Cambodian,category_Cantonese,category_Caribbean,category_Casinos,category_Caterers,category_Cheesesteaks,category_Chicken Shop,category_Chicken Wings,category_Chinese,category_Chocolatiers & Shops,category_Cocktail Bars,category_Coffee & Tea Supplies,category_Coffee & Tea,category_Coffee Roasteries,category_Coffeeshops,category_Colombian,category_Comfort Food,category_Creperies,category_Cuban,category_Cupcakes,category_Custom Cakes,category_Czech,category_Delis,category_Desserts,category_Dim Sum,category_Diners,category_Dinner Theater,category_Do-It-Yourself Food,category_Dominican,category_Donuts,category_Eatertainment,category_Ethiopian,category_Ethnic Food,category_Event Planning & Services,category_Falafel,category_Fast Food,category_Festivals,category_Filipino,category_Fish & Chips,category_Fondue,category_Food Court,category_Food Delivery Services,category_Food Stands,category_Food Trucks,category_Food,category_French,category_Gastropubs,category_German,category_Gluten-Free,category_Greek,category_Grocery,category_Halal,category_Hawaiian,category_Herbs & Spices,category_Hookah Bars,category_Hot Dogs,category_Hot Pot,category_Hotel bar,category_Hotels,category_Ice Cream & Frozen Yogurt,category_Imported Food,category_Indian,category_Indonesian,category_Irish Pub,category_Irish,category_Italian,category_Izakaya,category_Japanese,category_Juice Bars & Smoothies,category_Kebab,category_Korean,category_Kosher,category_Laotian,category_Latin American,category_Lebanese,category_Live/Raw Food,category_Local Flavor,category_Lounges,category_Macarons,category_Malaysian,category_Marinas,category_Meat Shops,category_Mediterranean,category_Mexican,category_Middle Eastern,category_Modern European,category_Mongolian,category_Moroccan,category_Music Venues,category_New Mexican Cuisine,category_Nightlife,category_Noodles,category_Pakistani,category_Pan Asian,category_Pasta Shops,category_Patisserie/Cake Shop,category_Persian/Iranian,category_Peruvian,category_Pizza,category_Polish,category_Popcorn Shops,category_Poutineries,category_Pretzels,category_Pubs,category_Puerto Rican,category_Ramen,category_Russian,category_Salad,category_Salvadoran,category_Sandwiches,category_Scandinavian,category_Seafood,category_Soul Food,category_Soup,category_Southern,category_Spanish,category_Specialty Food,category_Sports Bars,cat

In [87]:
businessDF.iloc[businessDF[businessDF['business_id']=='Rut5I04WZ2Hm2GEyDwk8YA']['postal_code'].index[0],5] = 85034
businessDF.iloc[businessDF[businessDF['business_id']=='DmyS9b7ykIOo7XwYt5I9wg']['postal_code'].index[0],5] = 85034
businessDF.iloc[businessDF[businessDF['business_id']=='NOUYmgX2Hl2BlhqsWCUAyg']['postal_code'].index[0],5] = 85339

In [88]:
businessDF[businessDF['business_id']=='Rut5I04WZ2Hm2GEyDwk8YA']

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,AgesAllowed,Alcohol,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,Ambience_upscale,BikeParking,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_lot,BusinessParking_street,BusinessParking_valet,BusinessParking_validated,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_halal,DietaryRestrictions_kosher,DietaryRestrictions_soy-free,DietaryRestrictions_vegan,DietaryRestrictions_vegetarian,DogsAllowed,DriveThru,GoodForKids,GoodForMeal_breakfast,GoodForMeal_brunch,GoodForMeal_dessert,GoodForMeal_dinner,GoodForMeal_latenight,GoodForMeal_lunch,HappyHour,HasTV,Music_no_music,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,category_Afghan,category_African,category_American (New),category_American (Traditional),category_Arabian,category_Arcades,category_Argentine,category_Armenian,category_Arts & Entertainment,category_Asian Fusion,category_Australian,category_Austrian,category_Bagels,category_Bakeries,category_Barbeque,category_Bars,category_Beer Bar,category_Beer Gardens,category_Beer,category_Belgian,category_Bistros,category_Brasseries,category_Brazilian,category_Breakfast & Brunch,category_Breweries,category_Brewpubs,category_British,category_Bubble Tea,category_Buffets,category_Burgers,category_Cafes,category_Cafeteria,category_Cajun/Creole,category_Cambodian,category_Cantonese,category_Caribbean,category_Casinos,category_Caterers,category_Cheesesteaks,category_Chicken Shop,category_Chicken Wings,category_Chinese,category_Chocolatiers & Shops,category_Cocktail Bars,category_Coffee & Tea Supplies,category_Coffee & Tea,category_Coffee Roasteries,category_Coffeeshops,category_Colombian,category_Comfort Food,category_Creperies,category_Cuban,category_Cupcakes,category_Custom Cakes,category_Czech,category_Delis,category_Desserts,category_Dim Sum,category_Diners,category_Dinner Theater,category_Do-It-Yourself Food,category_Dominican,category_Donuts,category_Eatertainment,category_Ethiopian,category_Ethnic Food,category_Event Planning & Services,category_Falafel,category_Fast Food,category_Festivals,category_Filipino,category_Fish & Chips,category_Fondue,category_Food Court,category_Food Delivery Services,category_Food Stands,category_Food Trucks,category_Food,category_French,category_Gastropubs,category_German,category_Gluten-Free,category_Greek,category_Grocery,category_Halal,category_Hawaiian,category_Herbs & Spices,category_Hookah Bars,category_Hot Dogs,category_Hot Pot,category_Hotel bar,category_Hotels,category_Ice Cream & Frozen Yogurt,category_Imported Food,category_Indian,category_Indonesian,category_Irish Pub,category_Irish,category_Italian,category_Izakaya,category_Japanese,category_Juice Bars & Smoothies,category_Kebab,category_Korean,category_Kosher,category_Laotian,category_Latin American,category_Lebanese,category_Live/Raw Food,category_Local Flavor,category_Lounges,category_Macarons,category_Malaysian,category_Marinas,category_Meat Shops,category_Mediterranean,category_Mexican,category_Middle Eastern,category_Modern European,category_Mongolian,category_Moroccan,category_Music Venues,category_New Mexican Cuisine,category_Nightlife,category_Noodles,category_Pakistani,category_Pan Asian,category_Pasta Shops,category_Patisserie/Cake Shop,category_Persian/Iranian,category_Peruvian,category_Pizza,category_Polish,category_Popcorn Shops,category_Poutineries,category_Pretzels,category_Pubs,category_Puerto Rican,category_Ramen,category_Russian,category_Salad,category_Salvadoran,category_Sandwiches,category_Scandinavian,category_Seafood,category_Soul Food,category_Soup,category_Southern,category_Spanish,category_Specialty Food,category_Sports Bars,cat

In [89]:
businessDF[businessDF['business_id']=='DmyS9b7ykIOo7XwYt5I9wg']

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,AgesAllowed,Alcohol,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,Ambience_upscale,BikeParking,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_lot,BusinessParking_street,BusinessParking_valet,BusinessParking_validated,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_halal,DietaryRestrictions_kosher,DietaryRestrictions_soy-free,DietaryRestrictions_vegan,DietaryRestrictions_vegetarian,DogsAllowed,DriveThru,GoodForKids,GoodForMeal_breakfast,GoodForMeal_brunch,GoodForMeal_dessert,GoodForMeal_dinner,GoodForMeal_latenight,GoodForMeal_lunch,HappyHour,HasTV,Music_no_music,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,category_Afghan,category_African,category_American (New),category_American (Traditional),category_Arabian,category_Arcades,category_Argentine,category_Armenian,category_Arts & Entertainment,category_Asian Fusion,category_Australian,category_Austrian,category_Bagels,category_Bakeries,category_Barbeque,category_Bars,category_Beer Bar,category_Beer Gardens,category_Beer,category_Belgian,category_Bistros,category_Brasseries,category_Brazilian,category_Breakfast & Brunch,category_Breweries,category_Brewpubs,category_British,category_Bubble Tea,category_Buffets,category_Burgers,category_Cafes,category_Cafeteria,category_Cajun/Creole,category_Cambodian,category_Cantonese,category_Caribbean,category_Casinos,category_Caterers,category_Cheesesteaks,category_Chicken Shop,category_Chicken Wings,category_Chinese,category_Chocolatiers & Shops,category_Cocktail Bars,category_Coffee & Tea Supplies,category_Coffee & Tea,category_Coffee Roasteries,category_Coffeeshops,category_Colombian,category_Comfort Food,category_Creperies,category_Cuban,category_Cupcakes,category_Custom Cakes,category_Czech,category_Delis,category_Desserts,category_Dim Sum,category_Diners,category_Dinner Theater,category_Do-It-Yourself Food,category_Dominican,category_Donuts,category_Eatertainment,category_Ethiopian,category_Ethnic Food,category_Event Planning & Services,category_Falafel,category_Fast Food,category_Festivals,category_Filipino,category_Fish & Chips,category_Fondue,category_Food Court,category_Food Delivery Services,category_Food Stands,category_Food Trucks,category_Food,category_French,category_Gastropubs,category_German,category_Gluten-Free,category_Greek,category_Grocery,category_Halal,category_Hawaiian,category_Herbs & Spices,category_Hookah Bars,category_Hot Dogs,category_Hot Pot,category_Hotel bar,category_Hotels,category_Ice Cream & Frozen Yogurt,category_Imported Food,category_Indian,category_Indonesian,category_Irish Pub,category_Irish,category_Italian,category_Izakaya,category_Japanese,category_Juice Bars & Smoothies,category_Kebab,category_Korean,category_Kosher,category_Laotian,category_Latin American,category_Lebanese,category_Live/Raw Food,category_Local Flavor,category_Lounges,category_Macarons,category_Malaysian,category_Marinas,category_Meat Shops,category_Mediterranean,category_Mexican,category_Middle Eastern,category_Modern European,category_Mongolian,category_Moroccan,category_Music Venues,category_New Mexican Cuisine,category_Nightlife,category_Noodles,category_Pakistani,category_Pan Asian,category_Pasta Shops,category_Patisserie/Cake Shop,category_Persian/Iranian,category_Peruvian,category_Pizza,category_Polish,category_Popcorn Shops,category_Poutineries,category_Pretzels,category_Pubs,category_Puerto Rican,category_Ramen,category_Russian,category_Salad,category_Salvadoran,category_Sandwiches,category_Scandinavian,category_Seafood,category_Soul Food,category_Soup,category_Southern,category_Spanish,category_Specialty Food,category_Sports Bars,cat

In [90]:
businessDF[businessDF['business_id']=='NOUYmgX2Hl2BlhqsWCUAyg']

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,AgesAllowed,Alcohol,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,Ambience_upscale,BikeParking,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_lot,BusinessParking_street,BusinessParking_valet,BusinessParking_validated,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_halal,DietaryRestrictions_kosher,DietaryRestrictions_soy-free,DietaryRestrictions_vegan,DietaryRestrictions_vegetarian,DogsAllowed,DriveThru,GoodForKids,GoodForMeal_breakfast,GoodForMeal_brunch,GoodForMeal_dessert,GoodForMeal_dinner,GoodForMeal_latenight,GoodForMeal_lunch,HappyHour,HasTV,Music_no_music,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,category_Afghan,category_African,category_American (New),category_American (Traditional),category_Arabian,category_Arcades,category_Argentine,category_Armenian,category_Arts & Entertainment,category_Asian Fusion,category_Australian,category_Austrian,category_Bagels,category_Bakeries,category_Barbeque,category_Bars,category_Beer Bar,category_Beer Gardens,category_Beer,category_Belgian,category_Bistros,category_Brasseries,category_Brazilian,category_Breakfast & Brunch,category_Breweries,category_Brewpubs,category_British,category_Bubble Tea,category_Buffets,category_Burgers,category_Cafes,category_Cafeteria,category_Cajun/Creole,category_Cambodian,category_Cantonese,category_Caribbean,category_Casinos,category_Caterers,category_Cheesesteaks,category_Chicken Shop,category_Chicken Wings,category_Chinese,category_Chocolatiers & Shops,category_Cocktail Bars,category_Coffee & Tea Supplies,category_Coffee & Tea,category_Coffee Roasteries,category_Coffeeshops,category_Colombian,category_Comfort Food,category_Creperies,category_Cuban,category_Cupcakes,category_Custom Cakes,category_Czech,category_Delis,category_Desserts,category_Dim Sum,category_Diners,category_Dinner Theater,category_Do-It-Yourself Food,category_Dominican,category_Donuts,category_Eatertainment,category_Ethiopian,category_Ethnic Food,category_Event Planning & Services,category_Falafel,category_Fast Food,category_Festivals,category_Filipino,category_Fish & Chips,category_Fondue,category_Food Court,category_Food Delivery Services,category_Food Stands,category_Food Trucks,category_Food,category_French,category_Gastropubs,category_German,category_Gluten-Free,category_Greek,category_Grocery,category_Halal,category_Hawaiian,category_Herbs & Spices,category_Hookah Bars,category_Hot Dogs,category_Hot Pot,category_Hotel bar,category_Hotels,category_Ice Cream & Frozen Yogurt,category_Imported Food,category_Indian,category_Indonesian,category_Irish Pub,category_Irish,category_Italian,category_Izakaya,category_Japanese,category_Juice Bars & Smoothies,category_Kebab,category_Korean,category_Kosher,category_Laotian,category_Latin American,category_Lebanese,category_Live/Raw Food,category_Local Flavor,category_Lounges,category_Macarons,category_Malaysian,category_Marinas,category_Meat Shops,category_Mediterranean,category_Mexican,category_Middle Eastern,category_Modern European,category_Mongolian,category_Moroccan,category_Music Venues,category_New Mexican Cuisine,category_Nightlife,category_Noodles,category_Pakistani,category_Pan Asian,category_Pasta Shops,category_Patisserie/Cake Shop,category_Persian/Iranian,category_Peruvian,category_Pizza,category_Polish,category_Popcorn Shops,category_Poutineries,category_Pretzels,category_Pubs,category_Puerto Rican,category_Ramen,category_Russian,category_Salad,category_Salvadoran,category_Sandwiches,category_Scandinavian,category_Seafood,category_Soul Food,category_Soup,category_Southern,category_Spanish,category_Specialty Food,category_Sports Bars,cat

In [91]:
businessDF.iloc[businessDF[businessDF['business_id']=='pgpnvPd3mWxI0O9WYUeLnA']['city'].index[0],3] = 'Glendale'

In [92]:
businessDF[businessDF['business_id']=='pgpnvPd3mWxI0O9WYUeLnA']['city']

4978    Glendale
Name: city, dtype: object

In [93]:
businessDF.iloc[businessDF[businessDF['business_id'] == 'RAUI7uwpMP6dlcHXkavwHg']['city'].index[0],3] = \
businessDF.iloc[businessDF[businessDF['business_id'] == 'zy6Hzao6lTmuJMZo4BemIg']['city'].index[0],3] = \
businessDF.iloc[businessDF[businessDF['business_id'] == '1M0Mo_ctYMTH0CTKgv4RpQ']['city'].index[0],3] = \
businessDF.iloc[businessDF[businessDF['business_id'] == 'MJoHqF2VIKhBuMzpRnwrZA']['city'].index[0],3] = \
businessDF.iloc[businessDF[businessDF['business_id'] == '81XqABPROlu8WIER-soVmA']['city'].index[0],3] = \
businessDF.iloc[businessDF[businessDF['business_id'] == 'qo4dzAKiwdN44wjexJWH4g']['city'].index[0],3] = \
businessDF.iloc[businessDF[businessDF['business_id'] == 'LYHSSljEdYktLfk3t1E3vg']['city'].index[0],3] = \
businessDF.iloc[businessDF[businessDF['business_id'] == '18NP-qL3VIJEBJGpns7aTw']['city'].index[0],3] = \
businessDF.iloc[businessDF[businessDF['business_id'] == '_XgvHKj5Mn8h9q61FrPkNg']['city'].index[0],3] = \
businessDF.iloc[businessDF[businessDF['business_id'] == 'NOUYmgX2Hl2BlhqsWCUAyg']['city'].index[0],3] = \
businessDF.iloc[businessDF[businessDF['business_id'] == '2THWm-2RT0MtkH-BKN6-FQ']['city'].index[0],3] = \
businessDF.iloc[businessDF[businessDF['business_id'] == '8jNrbT0weOHH43Lf3Bzyig']['city'].index[0],3] = \
businessDF.iloc[businessDF[businessDF['business_id'] == '0n2us6pzfJ4UK9qy1NWYOg']['city'].index[0],3] = \
businessDF.iloc[businessDF[businessDF['business_id'] == 'mDNh9K4eOIfjiZb1DlV88A']['city'].index[0],3] = \
businessDF.iloc[businessDF[businessDF['business_id'] == 'jWI_Ykdqcz9QNelFFktBEA']['city'].index[0],3] = \
businessDF.iloc[businessDF[businessDF['business_id'] == 'gaGhS-nnnjW_i6WwPWH75A']['city'].index[0],3] = \
businessDF.iloc[businessDF[businessDF['business_id'] == 'E54AvEdLfCFFkCvXmXJ7GA']['city'].index[0],3] = \
businessDF.iloc[businessDF[businessDF['business_id'] == '9mAMjgAoXrBwsNKaug-cmQ']['city'].index[0],3] = \
businessDF.iloc[businessDF[businessDF['business_id'] == '2CTHVcbajrh24zM-a6aKBg']['city'].index[0],3] = \
businessDF.iloc[businessDF[businessDF['business_id'] == 'KrCYn8fK6zg5JYu4O6kE2g']['city'].index[0],3] = \
businessDF.iloc[businessDF[businessDF['business_id'] == 'dGFrCnwIUEdi-zxkwDoezQ']['city'].index[0],3] = \
businessDF.iloc[businessDF[businessDF['business_id'] == '3rX4WqckBD2e4Hdeic9wUQ']['city'].index[0],3] = \
businessDF.iloc[businessDF[businessDF['business_id'] == 'Y-omMxOCDSYd7jbo1PSCig']['city'].index[0],3] = \
businessDF.iloc[businessDF[businessDF['business_id'] == 'nB0SDknanRBT0b9HDqJv8g']['city'].index[0],3] = \
businessDF.iloc[businessDF[businessDF['business_id'] == 'Oh1s-LpxUF9HswZ0zhGPPA']['city'].index[0],3] = \
businessDF.iloc[businessDF[businessDF['business_id'] == 'gOcq978lYnZaaRVGAqF69w']['city'].index[0],3] = \
businessDF.iloc[businessDF[businessDF['business_id'] == 'd2D09qs6bVs6Bjb9WnTIdA']['city'].index[0],3] = \
businessDF.iloc[businessDF[businessDF['business_id'] == '0oSWfLMJZnl2HhOc2lQb2g']['city'].index[0],3] = \
businessDF.iloc[businessDF[businessDF['business_id'] == '_8I19IRzDXmMSRES9cEGlw']['city'].index[0],3] = 'Laveen Village'

In [94]:
businessDF[businessDF['business_id'] == 'Oh1s-LpxUF9HswZ0zhGPPA']['city']

5897    Laveen Village
Name: city, dtype: object

In [95]:
businessDF.iloc[businessDF[businessDF['business_id']=='svU7GceMV2PsJe6MLSJWIA']['city'].index[0],3] = 'Mesa'

In [96]:
businessDF[businessDF['business_id']=='svU7GceMV2PsJe6MLSJWIA']

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,AgesAllowed,Alcohol,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,Ambience_upscale,BikeParking,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_lot,BusinessParking_street,BusinessParking_valet,BusinessParking_validated,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_halal,DietaryRestrictions_kosher,DietaryRestrictions_soy-free,DietaryRestrictions_vegan,DietaryRestrictions_vegetarian,DogsAllowed,DriveThru,GoodForKids,GoodForMeal_breakfast,GoodForMeal_brunch,GoodForMeal_dessert,GoodForMeal_dinner,GoodForMeal_latenight,GoodForMeal_lunch,HappyHour,HasTV,Music_no_music,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,category_Afghan,category_African,category_American (New),category_American (Traditional),category_Arabian,category_Arcades,category_Argentine,category_Armenian,category_Arts & Entertainment,category_Asian Fusion,category_Australian,category_Austrian,category_Bagels,category_Bakeries,category_Barbeque,category_Bars,category_Beer Bar,category_Beer Gardens,category_Beer,category_Belgian,category_Bistros,category_Brasseries,category_Brazilian,category_Breakfast & Brunch,category_Breweries,category_Brewpubs,category_British,category_Bubble Tea,category_Buffets,category_Burgers,category_Cafes,category_Cafeteria,category_Cajun/Creole,category_Cambodian,category_Cantonese,category_Caribbean,category_Casinos,category_Caterers,category_Cheesesteaks,category_Chicken Shop,category_Chicken Wings,category_Chinese,category_Chocolatiers & Shops,category_Cocktail Bars,category_Coffee & Tea Supplies,category_Coffee & Tea,category_Coffee Roasteries,category_Coffeeshops,category_Colombian,category_Comfort Food,category_Creperies,category_Cuban,category_Cupcakes,category_Custom Cakes,category_Czech,category_Delis,category_Desserts,category_Dim Sum,category_Diners,category_Dinner Theater,category_Do-It-Yourself Food,category_Dominican,category_Donuts,category_Eatertainment,category_Ethiopian,category_Ethnic Food,category_Event Planning & Services,category_Falafel,category_Fast Food,category_Festivals,category_Filipino,category_Fish & Chips,category_Fondue,category_Food Court,category_Food Delivery Services,category_Food Stands,category_Food Trucks,category_Food,category_French,category_Gastropubs,category_German,category_Gluten-Free,category_Greek,category_Grocery,category_Halal,category_Hawaiian,category_Herbs & Spices,category_Hookah Bars,category_Hot Dogs,category_Hot Pot,category_Hotel bar,category_Hotels,category_Ice Cream & Frozen Yogurt,category_Imported Food,category_Indian,category_Indonesian,category_Irish Pub,category_Irish,category_Italian,category_Izakaya,category_Japanese,category_Juice Bars & Smoothies,category_Kebab,category_Korean,category_Kosher,category_Laotian,category_Latin American,category_Lebanese,category_Live/Raw Food,category_Local Flavor,category_Lounges,category_Macarons,category_Malaysian,category_Marinas,category_Meat Shops,category_Mediterranean,category_Mexican,category_Middle Eastern,category_Modern European,category_Mongolian,category_Moroccan,category_Music Venues,category_New Mexican Cuisine,category_Nightlife,category_Noodles,category_Pakistani,category_Pan Asian,category_Pasta Shops,category_Patisserie/Cake Shop,category_Persian/Iranian,category_Peruvian,category_Pizza,category_Polish,category_Popcorn Shops,category_Poutineries,category_Pretzels,category_Pubs,category_Puerto Rican,category_Ramen,category_Russian,category_Salad,category_Salvadoran,category_Sandwiches,category_Scandinavian,category_Seafood,category_Soul Food,category_Soup,category_Southern,category_Spanish,category_Specialty Food,category_Sports Bars,cat

In [97]:
businessDF.iloc[businessDF[businessDF['business_id']=='mhjSXQR9LvL10SSHH6aHIg']['name'].index[0],1] = 'Mole'
businessDF.iloc[businessDF[businessDF['business_id']=='mhjSXQR9LvL10SSHH6aHIg']['city'].index[0],3] = 'Phoenix'

In [98]:
businessDF[businessDF['business_id']=='mhjSXQR9LvL10SSHH6aHIg']

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,AgesAllowed,Alcohol,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,Ambience_upscale,BikeParking,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_lot,BusinessParking_street,BusinessParking_valet,BusinessParking_validated,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_halal,DietaryRestrictions_kosher,DietaryRestrictions_soy-free,DietaryRestrictions_vegan,DietaryRestrictions_vegetarian,DogsAllowed,DriveThru,GoodForKids,GoodForMeal_breakfast,GoodForMeal_brunch,GoodForMeal_dessert,GoodForMeal_dinner,GoodForMeal_latenight,GoodForMeal_lunch,HappyHour,HasTV,Music_no_music,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,category_Afghan,category_African,category_American (New),category_American (Traditional),category_Arabian,category_Arcades,category_Argentine,category_Armenian,category_Arts & Entertainment,category_Asian Fusion,category_Australian,category_Austrian,category_Bagels,category_Bakeries,category_Barbeque,category_Bars,category_Beer Bar,category_Beer Gardens,category_Beer,category_Belgian,category_Bistros,category_Brasseries,category_Brazilian,category_Breakfast & Brunch,category_Breweries,category_Brewpubs,category_British,category_Bubble Tea,category_Buffets,category_Burgers,category_Cafes,category_Cafeteria,category_Cajun/Creole,category_Cambodian,category_Cantonese,category_Caribbean,category_Casinos,category_Caterers,category_Cheesesteaks,category_Chicken Shop,category_Chicken Wings,category_Chinese,category_Chocolatiers & Shops,category_Cocktail Bars,category_Coffee & Tea Supplies,category_Coffee & Tea,category_Coffee Roasteries,category_Coffeeshops,category_Colombian,category_Comfort Food,category_Creperies,category_Cuban,category_Cupcakes,category_Custom Cakes,category_Czech,category_Delis,category_Desserts,category_Dim Sum,category_Diners,category_Dinner Theater,category_Do-It-Yourself Food,category_Dominican,category_Donuts,category_Eatertainment,category_Ethiopian,category_Ethnic Food,category_Event Planning & Services,category_Falafel,category_Fast Food,category_Festivals,category_Filipino,category_Fish & Chips,category_Fondue,category_Food Court,category_Food Delivery Services,category_Food Stands,category_Food Trucks,category_Food,category_French,category_Gastropubs,category_German,category_Gluten-Free,category_Greek,category_Grocery,category_Halal,category_Hawaiian,category_Herbs & Spices,category_Hookah Bars,category_Hot Dogs,category_Hot Pot,category_Hotel bar,category_Hotels,category_Ice Cream & Frozen Yogurt,category_Imported Food,category_Indian,category_Indonesian,category_Irish Pub,category_Irish,category_Italian,category_Izakaya,category_Japanese,category_Juice Bars & Smoothies,category_Kebab,category_Korean,category_Kosher,category_Laotian,category_Latin American,category_Lebanese,category_Live/Raw Food,category_Local Flavor,category_Lounges,category_Macarons,category_Malaysian,category_Marinas,category_Meat Shops,category_Mediterranean,category_Mexican,category_Middle Eastern,category_Modern European,category_Mongolian,category_Moroccan,category_Music Venues,category_New Mexican Cuisine,category_Nightlife,category_Noodles,category_Pakistani,category_Pan Asian,category_Pasta Shops,category_Patisserie/Cake Shop,category_Persian/Iranian,category_Peruvian,category_Pizza,category_Polish,category_Popcorn Shops,category_Poutineries,category_Pretzels,category_Pubs,category_Puerto Rican,category_Ramen,category_Russian,category_Salad,category_Salvadoran,category_Sandwiches,category_Scandinavian,category_Seafood,category_Soul Food,category_Soup,category_Southern,category_Spanish,category_Specialty Food,category_Sports Bars,cat

In [99]:
businessDF.iloc[businessDF[businessDF['business_id']=='0Zu6KcjFJjBpCOjiOa2HvQ']['city'].index[0],3] = 'Scottsdale'
businessDF.iloc[businessDF[businessDF['business_id']=='JvHv4HDT7sU7rMYyDzJuMQ']['city'].index[0],3] = 'Scottsdale'
businessDF.iloc[businessDF[businessDF['business_id']=='Y3aGrZR8QBlj8C2l7oEvDg']['city'].index[0],3] = 'Scottsdale'

In [100]:
businessDF[businessDF['business_id']=='0Zu6KcjFJjBpCOjiOa2HvQ']

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,AgesAllowed,Alcohol,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,Ambience_upscale,BikeParking,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_lot,BusinessParking_street,BusinessParking_valet,BusinessParking_validated,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_halal,DietaryRestrictions_kosher,DietaryRestrictions_soy-free,DietaryRestrictions_vegan,DietaryRestrictions_vegetarian,DogsAllowed,DriveThru,GoodForKids,GoodForMeal_breakfast,GoodForMeal_brunch,GoodForMeal_dessert,GoodForMeal_dinner,GoodForMeal_latenight,GoodForMeal_lunch,HappyHour,HasTV,Music_no_music,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,category_Afghan,category_African,category_American (New),category_American (Traditional),category_Arabian,category_Arcades,category_Argentine,category_Armenian,category_Arts & Entertainment,category_Asian Fusion,category_Australian,category_Austrian,category_Bagels,category_Bakeries,category_Barbeque,category_Bars,category_Beer Bar,category_Beer Gardens,category_Beer,category_Belgian,category_Bistros,category_Brasseries,category_Brazilian,category_Breakfast & Brunch,category_Breweries,category_Brewpubs,category_British,category_Bubble Tea,category_Buffets,category_Burgers,category_Cafes,category_Cafeteria,category_Cajun/Creole,category_Cambodian,category_Cantonese,category_Caribbean,category_Casinos,category_Caterers,category_Cheesesteaks,category_Chicken Shop,category_Chicken Wings,category_Chinese,category_Chocolatiers & Shops,category_Cocktail Bars,category_Coffee & Tea Supplies,category_Coffee & Tea,category_Coffee Roasteries,category_Coffeeshops,category_Colombian,category_Comfort Food,category_Creperies,category_Cuban,category_Cupcakes,category_Custom Cakes,category_Czech,category_Delis,category_Desserts,category_Dim Sum,category_Diners,category_Dinner Theater,category_Do-It-Yourself Food,category_Dominican,category_Donuts,category_Eatertainment,category_Ethiopian,category_Ethnic Food,category_Event Planning & Services,category_Falafel,category_Fast Food,category_Festivals,category_Filipino,category_Fish & Chips,category_Fondue,category_Food Court,category_Food Delivery Services,category_Food Stands,category_Food Trucks,category_Food,category_French,category_Gastropubs,category_German,category_Gluten-Free,category_Greek,category_Grocery,category_Halal,category_Hawaiian,category_Herbs & Spices,category_Hookah Bars,category_Hot Dogs,category_Hot Pot,category_Hotel bar,category_Hotels,category_Ice Cream & Frozen Yogurt,category_Imported Food,category_Indian,category_Indonesian,category_Irish Pub,category_Irish,category_Italian,category_Izakaya,category_Japanese,category_Juice Bars & Smoothies,category_Kebab,category_Korean,category_Kosher,category_Laotian,category_Latin American,category_Lebanese,category_Live/Raw Food,category_Local Flavor,category_Lounges,category_Macarons,category_Malaysian,category_Marinas,category_Meat Shops,category_Mediterranean,category_Mexican,category_Middle Eastern,category_Modern European,category_Mongolian,category_Moroccan,category_Music Venues,category_New Mexican Cuisine,category_Nightlife,category_Noodles,category_Pakistani,category_Pan Asian,category_Pasta Shops,category_Patisserie/Cake Shop,category_Persian/Iranian,category_Peruvian,category_Pizza,category_Polish,category_Popcorn Shops,category_Poutineries,category_Pretzels,category_Pubs,category_Puerto Rican,category_Ramen,category_Russian,category_Salad,category_Salvadoran,category_Sandwiches,category_Scandinavian,category_Seafood,category_Soul Food,category_Soup,category_Southern,category_Spanish,category_Specialty Food,category_Sports Bars,cat

In [101]:
businessDF[businessDF['business_id']=='JvHv4HDT7sU7rMYyDzJuMQ']

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,AgesAllowed,Alcohol,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,Ambience_upscale,BikeParking,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_lot,BusinessParking_street,BusinessParking_valet,BusinessParking_validated,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_halal,DietaryRestrictions_kosher,DietaryRestrictions_soy-free,DietaryRestrictions_vegan,DietaryRestrictions_vegetarian,DogsAllowed,DriveThru,GoodForKids,GoodForMeal_breakfast,GoodForMeal_brunch,GoodForMeal_dessert,GoodForMeal_dinner,GoodForMeal_latenight,GoodForMeal_lunch,HappyHour,HasTV,Music_no_music,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,category_Afghan,category_African,category_American (New),category_American (Traditional),category_Arabian,category_Arcades,category_Argentine,category_Armenian,category_Arts & Entertainment,category_Asian Fusion,category_Australian,category_Austrian,category_Bagels,category_Bakeries,category_Barbeque,category_Bars,category_Beer Bar,category_Beer Gardens,category_Beer,category_Belgian,category_Bistros,category_Brasseries,category_Brazilian,category_Breakfast & Brunch,category_Breweries,category_Brewpubs,category_British,category_Bubble Tea,category_Buffets,category_Burgers,category_Cafes,category_Cafeteria,category_Cajun/Creole,category_Cambodian,category_Cantonese,category_Caribbean,category_Casinos,category_Caterers,category_Cheesesteaks,category_Chicken Shop,category_Chicken Wings,category_Chinese,category_Chocolatiers & Shops,category_Cocktail Bars,category_Coffee & Tea Supplies,category_Coffee & Tea,category_Coffee Roasteries,category_Coffeeshops,category_Colombian,category_Comfort Food,category_Creperies,category_Cuban,category_Cupcakes,category_Custom Cakes,category_Czech,category_Delis,category_Desserts,category_Dim Sum,category_Diners,category_Dinner Theater,category_Do-It-Yourself Food,category_Dominican,category_Donuts,category_Eatertainment,category_Ethiopian,category_Ethnic Food,category_Event Planning & Services,category_Falafel,category_Fast Food,category_Festivals,category_Filipino,category_Fish & Chips,category_Fondue,category_Food Court,category_Food Delivery Services,category_Food Stands,category_Food Trucks,category_Food,category_French,category_Gastropubs,category_German,category_Gluten-Free,category_Greek,category_Grocery,category_Halal,category_Hawaiian,category_Herbs & Spices,category_Hookah Bars,category_Hot Dogs,category_Hot Pot,category_Hotel bar,category_Hotels,category_Ice Cream & Frozen Yogurt,category_Imported Food,category_Indian,category_Indonesian,category_Irish Pub,category_Irish,category_Italian,category_Izakaya,category_Japanese,category_Juice Bars & Smoothies,category_Kebab,category_Korean,category_Kosher,category_Laotian,category_Latin American,category_Lebanese,category_Live/Raw Food,category_Local Flavor,category_Lounges,category_Macarons,category_Malaysian,category_Marinas,category_Meat Shops,category_Mediterranean,category_Mexican,category_Middle Eastern,category_Modern European,category_Mongolian,category_Moroccan,category_Music Venues,category_New Mexican Cuisine,category_Nightlife,category_Noodles,category_Pakistani,category_Pan Asian,category_Pasta Shops,category_Patisserie/Cake Shop,category_Persian/Iranian,category_Peruvian,category_Pizza,category_Polish,category_Popcorn Shops,category_Poutineries,category_Pretzels,category_Pubs,category_Puerto Rican,category_Ramen,category_Russian,category_Salad,category_Salvadoran,category_Sandwiches,category_Scandinavian,category_Seafood,category_Soul Food,category_Soup,category_Southern,category_Spanish,category_Specialty Food,category_Sports Bars,cat

In [102]:
businessDF[businessDF['business_id']=='Y3aGrZR8QBlj8C2l7oEvDg']

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,AgesAllowed,Alcohol,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,Ambience_upscale,BikeParking,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_lot,BusinessParking_street,BusinessParking_valet,BusinessParking_validated,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_halal,DietaryRestrictions_kosher,DietaryRestrictions_soy-free,DietaryRestrictions_vegan,DietaryRestrictions_vegetarian,DogsAllowed,DriveThru,GoodForKids,GoodForMeal_breakfast,GoodForMeal_brunch,GoodForMeal_dessert,GoodForMeal_dinner,GoodForMeal_latenight,GoodForMeal_lunch,HappyHour,HasTV,Music_no_music,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,category_Afghan,category_African,category_American (New),category_American (Traditional),category_Arabian,category_Arcades,category_Argentine,category_Armenian,category_Arts & Entertainment,category_Asian Fusion,category_Australian,category_Austrian,category_Bagels,category_Bakeries,category_Barbeque,category_Bars,category_Beer Bar,category_Beer Gardens,category_Beer,category_Belgian,category_Bistros,category_Brasseries,category_Brazilian,category_Breakfast & Brunch,category_Breweries,category_Brewpubs,category_British,category_Bubble Tea,category_Buffets,category_Burgers,category_Cafes,category_Cafeteria,category_Cajun/Creole,category_Cambodian,category_Cantonese,category_Caribbean,category_Casinos,category_Caterers,category_Cheesesteaks,category_Chicken Shop,category_Chicken Wings,category_Chinese,category_Chocolatiers & Shops,category_Cocktail Bars,category_Coffee & Tea Supplies,category_Coffee & Tea,category_Coffee Roasteries,category_Coffeeshops,category_Colombian,category_Comfort Food,category_Creperies,category_Cuban,category_Cupcakes,category_Custom Cakes,category_Czech,category_Delis,category_Desserts,category_Dim Sum,category_Diners,category_Dinner Theater,category_Do-It-Yourself Food,category_Dominican,category_Donuts,category_Eatertainment,category_Ethiopian,category_Ethnic Food,category_Event Planning & Services,category_Falafel,category_Fast Food,category_Festivals,category_Filipino,category_Fish & Chips,category_Fondue,category_Food Court,category_Food Delivery Services,category_Food Stands,category_Food Trucks,category_Food,category_French,category_Gastropubs,category_German,category_Gluten-Free,category_Greek,category_Grocery,category_Halal,category_Hawaiian,category_Herbs & Spices,category_Hookah Bars,category_Hot Dogs,category_Hot Pot,category_Hotel bar,category_Hotels,category_Ice Cream & Frozen Yogurt,category_Imported Food,category_Indian,category_Indonesian,category_Irish Pub,category_Irish,category_Italian,category_Izakaya,category_Japanese,category_Juice Bars & Smoothies,category_Kebab,category_Korean,category_Kosher,category_Laotian,category_Latin American,category_Lebanese,category_Live/Raw Food,category_Local Flavor,category_Lounges,category_Macarons,category_Malaysian,category_Marinas,category_Meat Shops,category_Mediterranean,category_Mexican,category_Middle Eastern,category_Modern European,category_Mongolian,category_Moroccan,category_Music Venues,category_New Mexican Cuisine,category_Nightlife,category_Noodles,category_Pakistani,category_Pan Asian,category_Pasta Shops,category_Patisserie/Cake Shop,category_Persian/Iranian,category_Peruvian,category_Pizza,category_Polish,category_Popcorn Shops,category_Poutineries,category_Pretzels,category_Pubs,category_Puerto Rican,category_Ramen,category_Russian,category_Salad,category_Salvadoran,category_Sandwiches,category_Scandinavian,category_Seafood,category_Soul Food,category_Soup,category_Southern,category_Spanish,category_Specialty Food,category_Sports Bars,cat

In [103]:
businessDF.iloc[businessDF[businessDF['business_id']=='boFrQQWEXyNe79kQ_DOjYA']['city'].index[0],3] = 'Sun City West'

In [104]:
businessDF[businessDF['business_id']=='boFrQQWEXyNe79kQ_DOjYA']['city']

4056    Sun City West
Name: city, dtype: object

In [105]:
businessDF[businessDF['business_id']=='WGQpjj6eA6mRBxp4XST6Ig']

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,AgesAllowed,Alcohol,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,Ambience_upscale,BikeParking,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_lot,BusinessParking_street,BusinessParking_valet,BusinessParking_validated,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_halal,DietaryRestrictions_kosher,DietaryRestrictions_soy-free,DietaryRestrictions_vegan,DietaryRestrictions_vegetarian,DogsAllowed,DriveThru,GoodForKids,GoodForMeal_breakfast,GoodForMeal_brunch,GoodForMeal_dessert,GoodForMeal_dinner,GoodForMeal_latenight,GoodForMeal_lunch,HappyHour,HasTV,Music_no_music,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,category_Afghan,category_African,category_American (New),category_American (Traditional),category_Arabian,category_Arcades,category_Argentine,category_Armenian,category_Arts & Entertainment,category_Asian Fusion,category_Australian,category_Austrian,category_Bagels,category_Bakeries,category_Barbeque,category_Bars,category_Beer Bar,category_Beer Gardens,category_Beer,category_Belgian,category_Bistros,category_Brasseries,category_Brazilian,category_Breakfast & Brunch,category_Breweries,category_Brewpubs,category_British,category_Bubble Tea,category_Buffets,category_Burgers,category_Cafes,category_Cafeteria,category_Cajun/Creole,category_Cambodian,category_Cantonese,category_Caribbean,category_Casinos,category_Caterers,category_Cheesesteaks,category_Chicken Shop,category_Chicken Wings,category_Chinese,category_Chocolatiers & Shops,category_Cocktail Bars,category_Coffee & Tea Supplies,category_Coffee & Tea,category_Coffee Roasteries,category_Coffeeshops,category_Colombian,category_Comfort Food,category_Creperies,category_Cuban,category_Cupcakes,category_Custom Cakes,category_Czech,category_Delis,category_Desserts,category_Dim Sum,category_Diners,category_Dinner Theater,category_Do-It-Yourself Food,category_Dominican,category_Donuts,category_Eatertainment,category_Ethiopian,category_Ethnic Food,category_Event Planning & Services,category_Falafel,category_Fast Food,category_Festivals,category_Filipino,category_Fish & Chips,category_Fondue,category_Food Court,category_Food Delivery Services,category_Food Stands,category_Food Trucks,category_Food,category_French,category_Gastropubs,category_German,category_Gluten-Free,category_Greek,category_Grocery,category_Halal,category_Hawaiian,category_Herbs & Spices,category_Hookah Bars,category_Hot Dogs,category_Hot Pot,category_Hotel bar,category_Hotels,category_Ice Cream & Frozen Yogurt,category_Imported Food,category_Indian,category_Indonesian,category_Irish Pub,category_Irish,category_Italian,category_Izakaya,category_Japanese,category_Juice Bars & Smoothies,category_Kebab,category_Korean,category_Kosher,category_Laotian,category_Latin American,category_Lebanese,category_Live/Raw Food,category_Local Flavor,category_Lounges,category_Macarons,category_Malaysian,category_Marinas,category_Meat Shops,category_Mediterranean,category_Mexican,category_Middle Eastern,category_Modern European,category_Mongolian,category_Moroccan,category_Music Venues,category_New Mexican Cuisine,category_Nightlife,category_Noodles,category_Pakistani,category_Pan Asian,category_Pasta Shops,category_Patisserie/Cake Shop,category_Persian/Iranian,category_Peruvian,category_Pizza,category_Polish,category_Popcorn Shops,category_Poutineries,category_Pretzels,category_Pubs,category_Puerto Rican,category_Ramen,category_Russian,category_Salad,category_Salvadoran,category_Sandwiches,category_Scandinavian,category_Seafood,category_Soul Food,category_Soup,category_Southern,category_Spanish,category_Specialty Food,category_Sports Bars,cat

In [106]:
businessDF= businessDF.drop(4104, axis=0)

In [107]:
businessDF[businessDF['business_id']=='WGQpjj6eA6mRBxp4XST6Ig']

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,AgesAllowed,Alcohol,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,Ambience_upscale,BikeParking,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_lot,BusinessParking_street,BusinessParking_valet,BusinessParking_validated,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_halal,DietaryRestrictions_kosher,DietaryRestrictions_soy-free,DietaryRestrictions_vegan,DietaryRestrictions_vegetarian,DogsAllowed,DriveThru,GoodForKids,GoodForMeal_breakfast,GoodForMeal_brunch,GoodForMeal_dessert,GoodForMeal_dinner,GoodForMeal_latenight,GoodForMeal_lunch,HappyHour,HasTV,Music_no_music,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,category_Afghan,category_African,category_American (New),category_American (Traditional),category_Arabian,category_Arcades,category_Argentine,category_Armenian,category_Arts & Entertainment,category_Asian Fusion,category_Australian,category_Austrian,category_Bagels,category_Bakeries,category_Barbeque,category_Bars,category_Beer Bar,category_Beer Gardens,category_Beer,category_Belgian,category_Bistros,category_Brasseries,category_Brazilian,category_Breakfast & Brunch,category_Breweries,category_Brewpubs,category_British,category_Bubble Tea,category_Buffets,category_Burgers,category_Cafes,category_Cafeteria,category_Cajun/Creole,category_Cambodian,category_Cantonese,category_Caribbean,category_Casinos,category_Caterers,category_Cheesesteaks,category_Chicken Shop,category_Chicken Wings,category_Chinese,category_Chocolatiers & Shops,category_Cocktail Bars,category_Coffee & Tea Supplies,category_Coffee & Tea,category_Coffee Roasteries,category_Coffeeshops,category_Colombian,category_Comfort Food,category_Creperies,category_Cuban,category_Cupcakes,category_Custom Cakes,category_Czech,category_Delis,category_Desserts,category_Dim Sum,category_Diners,category_Dinner Theater,category_Do-It-Yourself Food,category_Dominican,category_Donuts,category_Eatertainment,category_Ethiopian,category_Ethnic Food,category_Event Planning & Services,category_Falafel,category_Fast Food,category_Festivals,category_Filipino,category_Fish & Chips,category_Fondue,category_Food Court,category_Food Delivery Services,category_Food Stands,category_Food Trucks,category_Food,category_French,category_Gastropubs,category_German,category_Gluten-Free,category_Greek,category_Grocery,category_Halal,category_Hawaiian,category_Herbs & Spices,category_Hookah Bars,category_Hot Dogs,category_Hot Pot,category_Hotel bar,category_Hotels,category_Ice Cream & Frozen Yogurt,category_Imported Food,category_Indian,category_Indonesian,category_Irish Pub,category_Irish,category_Italian,category_Izakaya,category_Japanese,category_Juice Bars & Smoothies,category_Kebab,category_Korean,category_Kosher,category_Laotian,category_Latin American,category_Lebanese,category_Live/Raw Food,category_Local Flavor,category_Lounges,category_Macarons,category_Malaysian,category_Marinas,category_Meat Shops,category_Mediterranean,category_Mexican,category_Middle Eastern,category_Modern European,category_Mongolian,category_Moroccan,category_Music Venues,category_New Mexican Cuisine,category_Nightlife,category_Noodles,category_Pakistani,category_Pan Asian,category_Pasta Shops,category_Patisserie/Cake Shop,category_Persian/Iranian,category_Peruvian,category_Pizza,category_Polish,category_Popcorn Shops,category_Poutineries,category_Pretzels,category_Pubs,category_Puerto Rican,category_Ramen,category_Russian,category_Salad,category_Salvadoran,category_Sandwiches,category_Scandinavian,category_Seafood,category_Soul Food,category_Soup,category_Southern,category_Spanish,category_Specialty Food,category_Sports Bars,cat

In [108]:
businessDF= businessDF.drop(4973, axis=0)

In [109]:
businessDF[businessDF['business_id']=='82bfom6b5BQlnnd4m3wwew']

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,AgesAllowed,Alcohol,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,Ambience_upscale,BikeParking,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_lot,BusinessParking_street,BusinessParking_valet,BusinessParking_validated,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_halal,DietaryRestrictions_kosher,DietaryRestrictions_soy-free,DietaryRestrictions_vegan,DietaryRestrictions_vegetarian,DogsAllowed,DriveThru,GoodForKids,GoodForMeal_breakfast,GoodForMeal_brunch,GoodForMeal_dessert,GoodForMeal_dinner,GoodForMeal_latenight,GoodForMeal_lunch,HappyHour,HasTV,Music_no_music,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,category_Afghan,category_African,category_American (New),category_American (Traditional),category_Arabian,category_Arcades,category_Argentine,category_Armenian,category_Arts & Entertainment,category_Asian Fusion,category_Australian,category_Austrian,category_Bagels,category_Bakeries,category_Barbeque,category_Bars,category_Beer Bar,category_Beer Gardens,category_Beer,category_Belgian,category_Bistros,category_Brasseries,category_Brazilian,category_Breakfast & Brunch,category_Breweries,category_Brewpubs,category_British,category_Bubble Tea,category_Buffets,category_Burgers,category_Cafes,category_Cafeteria,category_Cajun/Creole,category_Cambodian,category_Cantonese,category_Caribbean,category_Casinos,category_Caterers,category_Cheesesteaks,category_Chicken Shop,category_Chicken Wings,category_Chinese,category_Chocolatiers & Shops,category_Cocktail Bars,category_Coffee & Tea Supplies,category_Coffee & Tea,category_Coffee Roasteries,category_Coffeeshops,category_Colombian,category_Comfort Food,category_Creperies,category_Cuban,category_Cupcakes,category_Custom Cakes,category_Czech,category_Delis,category_Desserts,category_Dim Sum,category_Diners,category_Dinner Theater,category_Do-It-Yourself Food,category_Dominican,category_Donuts,category_Eatertainment,category_Ethiopian,category_Ethnic Food,category_Event Planning & Services,category_Falafel,category_Fast Food,category_Festivals,category_Filipino,category_Fish & Chips,category_Fondue,category_Food Court,category_Food Delivery Services,category_Food Stands,category_Food Trucks,category_Food,category_French,category_Gastropubs,category_German,category_Gluten-Free,category_Greek,category_Grocery,category_Halal,category_Hawaiian,category_Herbs & Spices,category_Hookah Bars,category_Hot Dogs,category_Hot Pot,category_Hotel bar,category_Hotels,category_Ice Cream & Frozen Yogurt,category_Imported Food,category_Indian,category_Indonesian,category_Irish Pub,category_Irish,category_Italian,category_Izakaya,category_Japanese,category_Juice Bars & Smoothies,category_Kebab,category_Korean,category_Kosher,category_Laotian,category_Latin American,category_Lebanese,category_Live/Raw Food,category_Local Flavor,category_Lounges,category_Macarons,category_Malaysian,category_Marinas,category_Meat Shops,category_Mediterranean,category_Mexican,category_Middle Eastern,category_Modern European,category_Mongolian,category_Moroccan,category_Music Venues,category_New Mexican Cuisine,category_Nightlife,category_Noodles,category_Pakistani,category_Pan Asian,category_Pasta Shops,category_Patisserie/Cake Shop,category_Persian/Iranian,category_Peruvian,category_Pizza,category_Polish,category_Popcorn Shops,category_Poutineries,category_Pretzels,category_Pubs,category_Puerto Rican,category_Ramen,category_Russian,category_Salad,category_Salvadoran,category_Sandwiches,category_Scandinavian,category_Seafood,category_Soul Food,category_Soup,category_Southern,category_Spanish,category_Specialty Food,category_Sports Bars,cat

In [110]:
businessDF[businessDF['name']=='Desert Finds']

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,AgesAllowed,Alcohol,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,Ambience_upscale,BikeParking,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_lot,BusinessParking_street,BusinessParking_valet,BusinessParking_validated,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_halal,DietaryRestrictions_kosher,DietaryRestrictions_soy-free,DietaryRestrictions_vegan,DietaryRestrictions_vegetarian,DogsAllowed,DriveThru,GoodForKids,GoodForMeal_breakfast,GoodForMeal_brunch,GoodForMeal_dessert,GoodForMeal_dinner,GoodForMeal_latenight,GoodForMeal_lunch,HappyHour,HasTV,Music_no_music,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,category_Afghan,category_African,category_American (New),category_American (Traditional),category_Arabian,category_Arcades,category_Argentine,category_Armenian,category_Arts & Entertainment,category_Asian Fusion,category_Australian,category_Austrian,category_Bagels,category_Bakeries,category_Barbeque,category_Bars,category_Beer Bar,category_Beer Gardens,category_Beer,category_Belgian,category_Bistros,category_Brasseries,category_Brazilian,category_Breakfast & Brunch,category_Breweries,category_Brewpubs,category_British,category_Bubble Tea,category_Buffets,category_Burgers,category_Cafes,category_Cafeteria,category_Cajun/Creole,category_Cambodian,category_Cantonese,category_Caribbean,category_Casinos,category_Caterers,category_Cheesesteaks,category_Chicken Shop,category_Chicken Wings,category_Chinese,category_Chocolatiers & Shops,category_Cocktail Bars,category_Coffee & Tea Supplies,category_Coffee & Tea,category_Coffee Roasteries,category_Coffeeshops,category_Colombian,category_Comfort Food,category_Creperies,category_Cuban,category_Cupcakes,category_Custom Cakes,category_Czech,category_Delis,category_Desserts,category_Dim Sum,category_Diners,category_Dinner Theater,category_Do-It-Yourself Food,category_Dominican,category_Donuts,category_Eatertainment,category_Ethiopian,category_Ethnic Food,category_Event Planning & Services,category_Falafel,category_Fast Food,category_Festivals,category_Filipino,category_Fish & Chips,category_Fondue,category_Food Court,category_Food Delivery Services,category_Food Stands,category_Food Trucks,category_Food,category_French,category_Gastropubs,category_German,category_Gluten-Free,category_Greek,category_Grocery,category_Halal,category_Hawaiian,category_Herbs & Spices,category_Hookah Bars,category_Hot Dogs,category_Hot Pot,category_Hotel bar,category_Hotels,category_Ice Cream & Frozen Yogurt,category_Imported Food,category_Indian,category_Indonesian,category_Irish Pub,category_Irish,category_Italian,category_Izakaya,category_Japanese,category_Juice Bars & Smoothies,category_Kebab,category_Korean,category_Kosher,category_Laotian,category_Latin American,category_Lebanese,category_Live/Raw Food,category_Local Flavor,category_Lounges,category_Macarons,category_Malaysian,category_Marinas,category_Meat Shops,category_Mediterranean,category_Mexican,category_Middle Eastern,category_Modern European,category_Mongolian,category_Moroccan,category_Music Venues,category_New Mexican Cuisine,category_Nightlife,category_Noodles,category_Pakistani,category_Pan Asian,category_Pasta Shops,category_Patisserie/Cake Shop,category_Persian/Iranian,category_Peruvian,category_Pizza,category_Polish,category_Popcorn Shops,category_Poutineries,category_Pretzels,category_Pubs,category_Puerto Rican,category_Ramen,category_Russian,category_Salad,category_Salvadoran,category_Sandwiches,category_Scandinavian,category_Seafood,category_Soul Food,category_Soup,category_Southern,category_Spanish,category_Specialty Food,category_Sports Bars,cat

In [115]:
businessDF.iloc[businessDF[businessDF['business_id']=='gZGsReG0VeX4uKViHTB9EQ']['latitude'].index[0],6] = 33.5959024
businessDF.iloc[businessDF[businessDF['business_id']=='gZGsReG0VeX4uKViHTB9EQ']['longitude'].index[0],7] = -112.1531041

In [116]:
businessDF[businessDF['business_id']=='gZGsReG0VeX4uKViHTB9EQ']

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,AgesAllowed,Alcohol,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,Ambience_upscale,BikeParking,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_lot,BusinessParking_street,BusinessParking_valet,BusinessParking_validated,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_halal,DietaryRestrictions_kosher,DietaryRestrictions_soy-free,DietaryRestrictions_vegan,DietaryRestrictions_vegetarian,DogsAllowed,DriveThru,GoodForKids,GoodForMeal_breakfast,GoodForMeal_brunch,GoodForMeal_dessert,GoodForMeal_dinner,GoodForMeal_latenight,GoodForMeal_lunch,HappyHour,HasTV,Music_no_music,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,category_Afghan,category_African,category_American (New),category_American (Traditional),category_Arabian,category_Arcades,category_Argentine,category_Armenian,category_Arts & Entertainment,category_Asian Fusion,category_Australian,category_Austrian,category_Bagels,category_Bakeries,category_Barbeque,category_Bars,category_Beer Bar,category_Beer Gardens,category_Beer,category_Belgian,category_Bistros,category_Brasseries,category_Brazilian,category_Breakfast & Brunch,category_Breweries,category_Brewpubs,category_British,category_Bubble Tea,category_Buffets,category_Burgers,category_Cafes,category_Cafeteria,category_Cajun/Creole,category_Cambodian,category_Cantonese,category_Caribbean,category_Casinos,category_Caterers,category_Cheesesteaks,category_Chicken Shop,category_Chicken Wings,category_Chinese,category_Chocolatiers & Shops,category_Cocktail Bars,category_Coffee & Tea Supplies,category_Coffee & Tea,category_Coffee Roasteries,category_Coffeeshops,category_Colombian,category_Comfort Food,category_Creperies,category_Cuban,category_Cupcakes,category_Custom Cakes,category_Czech,category_Delis,category_Desserts,category_Dim Sum,category_Diners,category_Dinner Theater,category_Do-It-Yourself Food,category_Dominican,category_Donuts,category_Eatertainment,category_Ethiopian,category_Ethnic Food,category_Event Planning & Services,category_Falafel,category_Fast Food,category_Festivals,category_Filipino,category_Fish & Chips,category_Fondue,category_Food Court,category_Food Delivery Services,category_Food Stands,category_Food Trucks,category_Food,category_French,category_Gastropubs,category_German,category_Gluten-Free,category_Greek,category_Grocery,category_Halal,category_Hawaiian,category_Herbs & Spices,category_Hookah Bars,category_Hot Dogs,category_Hot Pot,category_Hotel bar,category_Hotels,category_Ice Cream & Frozen Yogurt,category_Imported Food,category_Indian,category_Indonesian,category_Irish Pub,category_Irish,category_Italian,category_Izakaya,category_Japanese,category_Juice Bars & Smoothies,category_Kebab,category_Korean,category_Kosher,category_Laotian,category_Latin American,category_Lebanese,category_Live/Raw Food,category_Local Flavor,category_Lounges,category_Macarons,category_Malaysian,category_Marinas,category_Meat Shops,category_Mediterranean,category_Mexican,category_Middle Eastern,category_Modern European,category_Mongolian,category_Moroccan,category_Music Venues,category_New Mexican Cuisine,category_Nightlife,category_Noodles,category_Pakistani,category_Pan Asian,category_Pasta Shops,category_Patisserie/Cake Shop,category_Persian/Iranian,category_Peruvian,category_Pizza,category_Polish,category_Popcorn Shops,category_Poutineries,category_Pretzels,category_Pubs,category_Puerto Rican,category_Ramen,category_Russian,category_Salad,category_Salvadoran,category_Sandwiches,category_Scandinavian,category_Seafood,category_Soul Food,category_Soup,category_Southern,category_Spanish,category_Specialty Food,category_Sports Bars,cat

In [117]:
businessDF[businessDF['business_id']=='kkEqZmVvVkgmCaOqE13mDg']['name']

655    #1 Sushi
Name: name, dtype: object

In [118]:
businessDF.iloc[businessDF[businessDF['business_id']=='kkEqZmVvVkgmCaOqE13mDg'].index[0],1]= '1 Sushi'

In [119]:
businessDF[businessDF['business_id']=='kkEqZmVvVkgmCaOqE13mDg']

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,AgesAllowed,Alcohol,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,Ambience_upscale,BikeParking,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_lot,BusinessParking_street,BusinessParking_valet,BusinessParking_validated,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_halal,DietaryRestrictions_kosher,DietaryRestrictions_soy-free,DietaryRestrictions_vegan,DietaryRestrictions_vegetarian,DogsAllowed,DriveThru,GoodForKids,GoodForMeal_breakfast,GoodForMeal_brunch,GoodForMeal_dessert,GoodForMeal_dinner,GoodForMeal_latenight,GoodForMeal_lunch,HappyHour,HasTV,Music_no_music,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,category_Afghan,category_African,category_American (New),category_American (Traditional),category_Arabian,category_Arcades,category_Argentine,category_Armenian,category_Arts & Entertainment,category_Asian Fusion,category_Australian,category_Austrian,category_Bagels,category_Bakeries,category_Barbeque,category_Bars,category_Beer Bar,category_Beer Gardens,category_Beer,category_Belgian,category_Bistros,category_Brasseries,category_Brazilian,category_Breakfast & Brunch,category_Breweries,category_Brewpubs,category_British,category_Bubble Tea,category_Buffets,category_Burgers,category_Cafes,category_Cafeteria,category_Cajun/Creole,category_Cambodian,category_Cantonese,category_Caribbean,category_Casinos,category_Caterers,category_Cheesesteaks,category_Chicken Shop,category_Chicken Wings,category_Chinese,category_Chocolatiers & Shops,category_Cocktail Bars,category_Coffee & Tea Supplies,category_Coffee & Tea,category_Coffee Roasteries,category_Coffeeshops,category_Colombian,category_Comfort Food,category_Creperies,category_Cuban,category_Cupcakes,category_Custom Cakes,category_Czech,category_Delis,category_Desserts,category_Dim Sum,category_Diners,category_Dinner Theater,category_Do-It-Yourself Food,category_Dominican,category_Donuts,category_Eatertainment,category_Ethiopian,category_Ethnic Food,category_Event Planning & Services,category_Falafel,category_Fast Food,category_Festivals,category_Filipino,category_Fish & Chips,category_Fondue,category_Food Court,category_Food Delivery Services,category_Food Stands,category_Food Trucks,category_Food,category_French,category_Gastropubs,category_German,category_Gluten-Free,category_Greek,category_Grocery,category_Halal,category_Hawaiian,category_Herbs & Spices,category_Hookah Bars,category_Hot Dogs,category_Hot Pot,category_Hotel bar,category_Hotels,category_Ice Cream & Frozen Yogurt,category_Imported Food,category_Indian,category_Indonesian,category_Irish Pub,category_Irish,category_Italian,category_Izakaya,category_Japanese,category_Juice Bars & Smoothies,category_Kebab,category_Korean,category_Kosher,category_Laotian,category_Latin American,category_Lebanese,category_Live/Raw Food,category_Local Flavor,category_Lounges,category_Macarons,category_Malaysian,category_Marinas,category_Meat Shops,category_Mediterranean,category_Mexican,category_Middle Eastern,category_Modern European,category_Mongolian,category_Moroccan,category_Music Venues,category_New Mexican Cuisine,category_Nightlife,category_Noodles,category_Pakistani,category_Pan Asian,category_Pasta Shops,category_Patisserie/Cake Shop,category_Persian/Iranian,category_Peruvian,category_Pizza,category_Polish,category_Popcorn Shops,category_Poutineries,category_Pretzels,category_Pubs,category_Puerto Rican,category_Ramen,category_Russian,category_Salad,category_Salvadoran,category_Sandwiches,category_Scandinavian,category_Seafood,category_Soul Food,category_Soup,category_Southern,category_Spanish,category_Specialty Food,category_Sports Bars,cat

In [120]:
businessDF[businessDF['business_id']=='ysaGG0hm7-ug-IuRd-8_ew']['name']

1308    #1 Fried Rice
Name: name, dtype: object

In [121]:
businessDF.iloc[businessDF[businessDF['business_id']=='ysaGG0hm7-ug-IuRd-8_ew'].index[0],1]= '1 Fried Rice'

In [122]:
businessDF[businessDF['business_id']=='ysaGG0hm7-ug-IuRd-8_ew']

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,AgesAllowed,Alcohol,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,Ambience_upscale,BikeParking,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_lot,BusinessParking_street,BusinessParking_valet,BusinessParking_validated,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_halal,DietaryRestrictions_kosher,DietaryRestrictions_soy-free,DietaryRestrictions_vegan,DietaryRestrictions_vegetarian,DogsAllowed,DriveThru,GoodForKids,GoodForMeal_breakfast,GoodForMeal_brunch,GoodForMeal_dessert,GoodForMeal_dinner,GoodForMeal_latenight,GoodForMeal_lunch,HappyHour,HasTV,Music_no_music,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,category_Afghan,category_African,category_American (New),category_American (Traditional),category_Arabian,category_Arcades,category_Argentine,category_Armenian,category_Arts & Entertainment,category_Asian Fusion,category_Australian,category_Austrian,category_Bagels,category_Bakeries,category_Barbeque,category_Bars,category_Beer Bar,category_Beer Gardens,category_Beer,category_Belgian,category_Bistros,category_Brasseries,category_Brazilian,category_Breakfast & Brunch,category_Breweries,category_Brewpubs,category_British,category_Bubble Tea,category_Buffets,category_Burgers,category_Cafes,category_Cafeteria,category_Cajun/Creole,category_Cambodian,category_Cantonese,category_Caribbean,category_Casinos,category_Caterers,category_Cheesesteaks,category_Chicken Shop,category_Chicken Wings,category_Chinese,category_Chocolatiers & Shops,category_Cocktail Bars,category_Coffee & Tea Supplies,category_Coffee & Tea,category_Coffee Roasteries,category_Coffeeshops,category_Colombian,category_Comfort Food,category_Creperies,category_Cuban,category_Cupcakes,category_Custom Cakes,category_Czech,category_Delis,category_Desserts,category_Dim Sum,category_Diners,category_Dinner Theater,category_Do-It-Yourself Food,category_Dominican,category_Donuts,category_Eatertainment,category_Ethiopian,category_Ethnic Food,category_Event Planning & Services,category_Falafel,category_Fast Food,category_Festivals,category_Filipino,category_Fish & Chips,category_Fondue,category_Food Court,category_Food Delivery Services,category_Food Stands,category_Food Trucks,category_Food,category_French,category_Gastropubs,category_German,category_Gluten-Free,category_Greek,category_Grocery,category_Halal,category_Hawaiian,category_Herbs & Spices,category_Hookah Bars,category_Hot Dogs,category_Hot Pot,category_Hotel bar,category_Hotels,category_Ice Cream & Frozen Yogurt,category_Imported Food,category_Indian,category_Indonesian,category_Irish Pub,category_Irish,category_Italian,category_Izakaya,category_Japanese,category_Juice Bars & Smoothies,category_Kebab,category_Korean,category_Kosher,category_Laotian,category_Latin American,category_Lebanese,category_Live/Raw Food,category_Local Flavor,category_Lounges,category_Macarons,category_Malaysian,category_Marinas,category_Meat Shops,category_Mediterranean,category_Mexican,category_Middle Eastern,category_Modern European,category_Mongolian,category_Moroccan,category_Music Venues,category_New Mexican Cuisine,category_Nightlife,category_Noodles,category_Pakistani,category_Pan Asian,category_Pasta Shops,category_Patisserie/Cake Shop,category_Persian/Iranian,category_Peruvian,category_Pizza,category_Polish,category_Popcorn Shops,category_Poutineries,category_Pretzels,category_Pubs,category_Puerto Rican,category_Ramen,category_Russian,category_Salad,category_Salvadoran,category_Sandwiches,category_Scandinavian,category_Seafood,category_Soul Food,category_Soup,category_Southern,category_Spanish,category_Specialty Food,category_Sports Bars,cat

In [123]:
businessDF[businessDF['business_id']=='4z-QW_f3RwCAxHB5fd58TA']['name']

6801    #1Brothers Pizza
Name: name, dtype: object

In [124]:
businessDF.iloc[businessDF[businessDF['business_id']=='4z-QW_f3RwCAxHB5fd58TA'].index[0],1]= '1 Brother\'s Pizza'

In [125]:
businessDF[businessDF['business_id']=='4z-QW_f3RwCAxHB5fd58TA']

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,AgesAllowed,Alcohol,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,Ambience_upscale,BikeParking,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_lot,BusinessParking_street,BusinessParking_valet,BusinessParking_validated,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_halal,DietaryRestrictions_kosher,DietaryRestrictions_soy-free,DietaryRestrictions_vegan,DietaryRestrictions_vegetarian,DogsAllowed,DriveThru,GoodForKids,GoodForMeal_breakfast,GoodForMeal_brunch,GoodForMeal_dessert,GoodForMeal_dinner,GoodForMeal_latenight,GoodForMeal_lunch,HappyHour,HasTV,Music_no_music,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,category_Afghan,category_African,category_American (New),category_American (Traditional),category_Arabian,category_Arcades,category_Argentine,category_Armenian,category_Arts & Entertainment,category_Asian Fusion,category_Australian,category_Austrian,category_Bagels,category_Bakeries,category_Barbeque,category_Bars,category_Beer Bar,category_Beer Gardens,category_Beer,category_Belgian,category_Bistros,category_Brasseries,category_Brazilian,category_Breakfast & Brunch,category_Breweries,category_Brewpubs,category_British,category_Bubble Tea,category_Buffets,category_Burgers,category_Cafes,category_Cafeteria,category_Cajun/Creole,category_Cambodian,category_Cantonese,category_Caribbean,category_Casinos,category_Caterers,category_Cheesesteaks,category_Chicken Shop,category_Chicken Wings,category_Chinese,category_Chocolatiers & Shops,category_Cocktail Bars,category_Coffee & Tea Supplies,category_Coffee & Tea,category_Coffee Roasteries,category_Coffeeshops,category_Colombian,category_Comfort Food,category_Creperies,category_Cuban,category_Cupcakes,category_Custom Cakes,category_Czech,category_Delis,category_Desserts,category_Dim Sum,category_Diners,category_Dinner Theater,category_Do-It-Yourself Food,category_Dominican,category_Donuts,category_Eatertainment,category_Ethiopian,category_Ethnic Food,category_Event Planning & Services,category_Falafel,category_Fast Food,category_Festivals,category_Filipino,category_Fish & Chips,category_Fondue,category_Food Court,category_Food Delivery Services,category_Food Stands,category_Food Trucks,category_Food,category_French,category_Gastropubs,category_German,category_Gluten-Free,category_Greek,category_Grocery,category_Halal,category_Hawaiian,category_Herbs & Spices,category_Hookah Bars,category_Hot Dogs,category_Hot Pot,category_Hotel bar,category_Hotels,category_Ice Cream & Frozen Yogurt,category_Imported Food,category_Indian,category_Indonesian,category_Irish Pub,category_Irish,category_Italian,category_Izakaya,category_Japanese,category_Juice Bars & Smoothies,category_Kebab,category_Korean,category_Kosher,category_Laotian,category_Latin American,category_Lebanese,category_Live/Raw Food,category_Local Flavor,category_Lounges,category_Macarons,category_Malaysian,category_Marinas,category_Meat Shops,category_Mediterranean,category_Mexican,category_Middle Eastern,category_Modern European,category_Mongolian,category_Moroccan,category_Music Venues,category_New Mexican Cuisine,category_Nightlife,category_Noodles,category_Pakistani,category_Pan Asian,category_Pasta Shops,category_Patisserie/Cake Shop,category_Persian/Iranian,category_Peruvian,category_Pizza,category_Polish,category_Popcorn Shops,category_Poutineries,category_Pretzels,category_Pubs,category_Puerto Rican,category_Ramen,category_Russian,category_Salad,category_Salvadoran,category_Sandwiches,category_Scandinavian,category_Seafood,category_Soul Food,category_Soup,category_Southern,category_Spanish,category_Specialty Food,category_Sports Bars,cat

In [126]:
businessDF.iloc[businessDF[businessDF['business_id']=='gWY94C5EgNClUOwqJXrBGg'].index[0],1] = '1 Brother\'s Pizza'

In [127]:
businessDF[businessDF['business_id']=='gWY94C5EgNClUOwqJXrBGg']

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,AgesAllowed,Alcohol,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,Ambience_upscale,BikeParking,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_lot,BusinessParking_street,BusinessParking_valet,BusinessParking_validated,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_halal,DietaryRestrictions_kosher,DietaryRestrictions_soy-free,DietaryRestrictions_vegan,DietaryRestrictions_vegetarian,DogsAllowed,DriveThru,GoodForKids,GoodForMeal_breakfast,GoodForMeal_brunch,GoodForMeal_dessert,GoodForMeal_dinner,GoodForMeal_latenight,GoodForMeal_lunch,HappyHour,HasTV,Music_no_music,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,category_Afghan,category_African,category_American (New),category_American (Traditional),category_Arabian,category_Arcades,category_Argentine,category_Armenian,category_Arts & Entertainment,category_Asian Fusion,category_Australian,category_Austrian,category_Bagels,category_Bakeries,category_Barbeque,category_Bars,category_Beer Bar,category_Beer Gardens,category_Beer,category_Belgian,category_Bistros,category_Brasseries,category_Brazilian,category_Breakfast & Brunch,category_Breweries,category_Brewpubs,category_British,category_Bubble Tea,category_Buffets,category_Burgers,category_Cafes,category_Cafeteria,category_Cajun/Creole,category_Cambodian,category_Cantonese,category_Caribbean,category_Casinos,category_Caterers,category_Cheesesteaks,category_Chicken Shop,category_Chicken Wings,category_Chinese,category_Chocolatiers & Shops,category_Cocktail Bars,category_Coffee & Tea Supplies,category_Coffee & Tea,category_Coffee Roasteries,category_Coffeeshops,category_Colombian,category_Comfort Food,category_Creperies,category_Cuban,category_Cupcakes,category_Custom Cakes,category_Czech,category_Delis,category_Desserts,category_Dim Sum,category_Diners,category_Dinner Theater,category_Do-It-Yourself Food,category_Dominican,category_Donuts,category_Eatertainment,category_Ethiopian,category_Ethnic Food,category_Event Planning & Services,category_Falafel,category_Fast Food,category_Festivals,category_Filipino,category_Fish & Chips,category_Fondue,category_Food Court,category_Food Delivery Services,category_Food Stands,category_Food Trucks,category_Food,category_French,category_Gastropubs,category_German,category_Gluten-Free,category_Greek,category_Grocery,category_Halal,category_Hawaiian,category_Herbs & Spices,category_Hookah Bars,category_Hot Dogs,category_Hot Pot,category_Hotel bar,category_Hotels,category_Ice Cream & Frozen Yogurt,category_Imported Food,category_Indian,category_Indonesian,category_Irish Pub,category_Irish,category_Italian,category_Izakaya,category_Japanese,category_Juice Bars & Smoothies,category_Kebab,category_Korean,category_Kosher,category_Laotian,category_Latin American,category_Lebanese,category_Live/Raw Food,category_Local Flavor,category_Lounges,category_Macarons,category_Malaysian,category_Marinas,category_Meat Shops,category_Mediterranean,category_Mexican,category_Middle Eastern,category_Modern European,category_Mongolian,category_Moroccan,category_Music Venues,category_New Mexican Cuisine,category_Nightlife,category_Noodles,category_Pakistani,category_Pan Asian,category_Pasta Shops,category_Patisserie/Cake Shop,category_Persian/Iranian,category_Peruvian,category_Pizza,category_Polish,category_Popcorn Shops,category_Poutineries,category_Pretzels,category_Pubs,category_Puerto Rican,category_Ramen,category_Russian,category_Salad,category_Salvadoran,category_Sandwiches,category_Scandinavian,category_Seafood,category_Soul Food,category_Soup,category_Southern,category_Spanish,category_Specialty Food,category_Sports Bars,cat

For confirmation and cross verification running the loop again to determine if any missing values' needs to be handled.

In [128]:
for colmn in businessDF.columns.values:
    print('Column name: ',colmn)
    print('No of Unique values: ',businessDF[colmn].nunique())
    print('Unique values: ',businessDF[colmn].unique())
    print('NaN count: ',businessDF[colmn].isnull().sum())
    if(colmn not in ['business_id','name','neighborhood','address','city','state','postal_code','latitude','longitude',
                     'stars','review_count']):
        print('Attribute counts:\n',businessDF[colmn].value_counts(),'\n')
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

Column name:  business_id
No of Unique values:  7365
Unique values:  ['rDMptJYWtnMhpQu_rRXHng' '1WBkAuQg81kokZIPMpn9Zg'
 'iPa__LOhse-hobC2Xmp-Kw' ... '5zva2MTtB5IX6TaoVLL-NA'
 'Gr-2oBg4XyduSKbvnE-i9g' 'UdEmYOnk2iJDY9lpEPAlJQ']
NaN count:  0
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  name
No of Unique values:  4168
Unique values:  ["McDonald's" 'Charr An American Burger Bar' 'Little Caesars Pizza' ...
 "Yin's Chinese Resturant" 'Zorbas Grill'
 'Salt & Lime Modern Mexican Grill']
NaN count:  0
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  address
No of Unique values:  6720
Unique values:  ['719 E Thunderbird Rd' '777 E Thunderbird Rd, Ste 107'
 '1635 E Camelback Rd' ... '1950 W Indian School Rd'
 '9397 E Shea Blvd, Ste 115' '590 N Alma School Rd, Ste 35']
NaN count:  85
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  city
No of Unique values:  31
U

NaN count:  0
Attribute counts:
 0    7362
1       3
Name: DietaryRestrictions_halal, dtype: int64 

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  DietaryRestrictions_kosher
No of Unique values:  2
Unique values:  [0 1]
NaN count:  0
Attribute counts:
 0    7362
1       3
Name: DietaryRestrictions_kosher, dtype: int64 

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  DietaryRestrictions_soy-free
No of Unique values:  2
Unique values:  [0 1]
NaN count:  0
Attribute counts:
 0    7362
1       3
Name: DietaryRestrictions_soy-free, dtype: int64 

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  DietaryRestrictions_vegan
No of Unique values:  2
Unique values:  [0 1]
NaN count:  0
Attribute counts:
 0    7353
1      12
Name: DietaryRestrictions_vegan, dtype: int64 

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  

Column name:  category_German
No of Unique values:  2
Unique values:  [0 1]
NaN count:  0
Attribute counts:
 0    7360
1       5
Name: category_German, dtype: int64 

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  category_Gluten-Free
No of Unique values:  2
Unique values:  [0 1]
NaN count:  0
Attribute counts:
 0    7217
1     148
Name: category_Gluten-Free, dtype: int64 

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  category_Greek
No of Unique values:  2
Unique values:  [0 1]
NaN count:  0
Attribute counts:
 0    7241
1     124
Name: category_Greek, dtype: int64 

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column name:  category_Grocery
No of Unique values:  2
Unique values:  [0 1]
NaN count:  0
Attribute counts:
 0    7303
1      62
Name: category_Grocery, dtype: int64 

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Column

In [129]:
businessDF.shape

(7365, 235)

Our restaurant dataframe has now finally 7365 businesses with 235 unique attributes.

Exporting the dataframe to a 'csv' file for Recommendation and Tableau EDA.

In [130]:
import csv

In [131]:
businessDF.to_csv('AZ_Restaurant_Final_Clean_Data.csv', header=True, encoding='utf-8', index=False)

## The resulting file will be used for further EDA using univariate/bivariate analysis and NLP/Recommendation processing. 